In [1]:
from flask import Flask, request, render_template,jsonify,redirect,make_response
import pymongo
from bson import json_util
import json
from bson.objectid import ObjectId
import time
import os

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Random import get_random_bytes
import gridfs
import datetime
import werkzeug
from flask_cors import CORS, cross_origin
import threading
from punctuator import Punctuator  
p = Punctuator('E:/Model/Demo-Europarl-EN.pcl')


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
E:\Anaconda\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Initialization

In [2]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

UPLOAD_FOLDER = 'E:/EduScribe-content_management/Upload'
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.pdf', '.docx']
app.config['MAX_CONTENT_LENGTH'] = 25*1000000 # 25 MB Limit
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# RSA 4096 bits encryption with e=65537

pub_key_file = open('public_key.pem','r')
pvt_key_file = open('private_key.pem','r')
pubkey = RSA.importKey(pub_key_file.read())
pvtkey = RSA.importKey(pvt_key_file.read())
encryptor = PKCS1_OAEP.new(pubkey)
decryptor = PKCS1_OAEP.new(pvtkey)

def decryption(encrypted_message):
    return decryptor.decrypt(encrypted_message).decode("utf-8")

def encryption(plain_message):
    return encryptor.encrypt(bytes(plain_message,'utf-8'))

# Test Area

In [3]:
@app.route('/student/pdf', methods = ['POST'])
def file_upload():
    if request.method== 'POST':
        files_get=request.files['pdf']
        filename = werkzeug.utils.secure_filename(files_get.filename)
        files_get.save(filename)
        print(files_get)
        response_string = {"status": "success"}
    else:
        response_string = {"status": "failed"}
    return json.dumps(response_string, indent=4)

# Welcome API

In [4]:
@app.route('/')
def welcome():
    return "<b>Welcome to Jakstech API Service!</b>"

In [5]:
@app.route('/recorder')
def recorder():
    return render_template('demofile.html')

# ANSWER PRINT

In [6]:
@app.route('/answerprint')
def answer_preview():
    return render_template('answer-print-preview.html')

In [7]:
@app.route('/answer-print-preview')
def answerpreview_js():
    return render_template('answer-print-preview.js')

In [8]:
@app.route('/answer-parser')
def answer_parser():
    return render_template('answer-parser.js')

In [9]:
@app.route('/assignment-view')
def assignment_view():
    return render_template('assignment-print-preview.html')

# Route files

In [10]:
%run -i admin-dashboard-files.py

In [11]:
%run -i teacher-dashboard-files.py

In [12]:
%run -i examiner-dashboard-files.py

In [13]:
%run -i proctor-dashboard-files.py

# Teacher Notes files

In [14]:
%run -i teachernotes-main-files.py

In [15]:
%run -i teachernotes-grammar-files.py

In [16]:
%run -i teachernotes-prose-files.py

In [17]:
%run -i teachernotes-poem-files.py

In [18]:
%run -i teachernotes-supplementary-files.py

In [19]:
%run -i prose-teacher-notes.py

In [20]:
%run -i grammar-teacher-notes.py

In [21]:
%run -i poem-teacher-notes.py

In [22]:
%run -i supplementary-teacher-notes.py

In [23]:
%run -i englishIIPaper10.py

# Login API

In [24]:
# check if student with given regno present 
@app.route('/student/check',methods=["POST"])
def student_regno_check():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [25]:
@app.route('/student/login-reattempt',methods=["POST"])
def student_login_reattempt():
    db = myclient["accounts"]
    collection = db["student"]
#     collection_accounts = db["students_profile"]
    if request.method == 'POST':
        regno = request.form['regno']
        dob = request.form['dob']
        db_response = collection.find_one({"regno": regno})
        if db_response:
#             db_response_account = collection_accounts.find_one({"parentId":db_response['_id'],"dob": dob})
            if dob == db_response["dob"]:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                return json.dumps({"status": "success","response": "ok"})
            else:
                return json.dumps({"status": "failed","response": "wrong date of birth"})
        else:
            return json.dumps({"status": "failed","response": "User doesn't exist"})

In [26]:
@app.route('/student/aadharcheck',methods=["POST"])
def student_aadhar_check():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        aadhar = request.form['aadhar']
        db_response = collection.find_one({"aadhar": aadhar})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [27]:
@app.route('/student/phnocheck',methods=["POST"])
def student_phno_check():
    db = myclient["accounts"]
    collection = db["students_profile"]
    if request.method == 'POST':
        mobile = request.form['mobile']
        db_response = collection.find_one({"mobile": mobile})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [28]:
@app.route('/student/login',methods=["POST"])
def student_login():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        passw = request.form['password']
        db_response = collection.find_one({"regno": regno,"password": passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == decryption(db_response['encrypted_pwd']):
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [29]:
@app.route('/student/logout',methods=["POST"])
def stu_logout():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
    
                if update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are not logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [30]:
@app.route('/teacher/check',methods=["POST"])
def teacher_email_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = collection.find_one({"regId": regId})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [31]:
@app.route('/teacher/aadharcheck',methods=["POST"])
def teacher_aadhar_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        aadhar = request.form['aadhar']
        db_response = collection.find_one({"aadhar": aadhar})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [32]:
@app.route('/teacher/phnocheck',methods=["POST"])
def teacher_phno_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        mobile = request.form['mobile']
        db_response = collection.find_one({"mobile": mobile})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [33]:
@app.route('/teacher/login-reattempt',methods=["POST"])
def teacher_login_reattempt():
    db = myclient["accounts"]
    collection = db["teacher"]

    if request.method == 'POST':
        regId = request.form['regId']
        dob = request.form['dob']
        db_response = collection.find_one({"regId": regId ,"dob": dob})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if dob == db_response['dob']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong date of birth"})
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [34]:
@app.route('/teacher/login',methods=["POST"])
def teacher_login():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['isEnrolled'] == True:
                if db_response['loggedIn'] != True:
                    if passw == decryption(db_response['encrypted_pwd']):
                        update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                        if update_result.acknowledged == True and update_result.modified_count == 1:
                            response_string = {"status": "success","response":"ok"}
                        else:
                            response_string = {"status":"failed","response":"database error"}
                        return json.dumps(response_string, indent=4)
                    else:
                        return json.dumps({"status": "failed","response":"wrong password"})
                else:
                    return json.dumps({"status": "failed","response": "You are logged in on another device"})
            else:
                return json.dumps({"status": "failed","response": "You are not approved by admin"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Forgot password - Email integration

In [35]:
%run -i student-forgot-password.py

In [36]:
%run -i teacher-forgot-password.py

In [37]:
%run -i admin-forgot-password.py

In [38]:
%run -i proctor-forgot-password.py

In [39]:
%run -i examiner-forgot-password.py

# ADMIN PROFILE

In [40]:
@app.route('/admin/profile',methods=["GET", "POST"])
def view_admin_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['admin_id']
        db_response = (collection.find_one({"admin_id":adminId},{"_id":0,"encrypted_pwd":0}))
       
        if db_response != None:
            return json.dumps(db_response)
        return json.dumps({"status": "failed","response": "User not found"})

In [41]:
@app.route('/admin/check',methods=["POST"])
def admin_id_check():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        admin_id = request.form['admin_id']
        db_response = collection.find_one({"admin_id": admin_id})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [42]:
# @app.route('/admin/password-check',methods=["POST"])
# def admin_password_check():
#     db = myclient["accounts"]
#     collection = db["admin"]
#     if request.method == 'POST':
#         admin_id = request.form['admin_id']
#         password = request.form['password']
#         encrypted = encryption(password)
#         db_response = collection.find_one({"admin_id": admin_id,"password":password,"encrypted_pwd":encrypted})
#         if db_response != None:
#             return json.dumps({"status": "success","response": "ok"})
#         else:
#             return json.dumps({"status": "failed","response": "User not found"})

In [43]:
@app.route('/admin/login-reattempt',methods=["POST"])
def admin_login_reattempt():
    db = myclient["accounts"]
    collection = db["admin"]

    if request.method == 'POST':
        admin_id = request.form['admin_id']
        dob = request.form['dob']
        db_response = collection.find_one({"admin_id":admin_id,"dob": dob})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if dob == db_response['dob']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong date of birth"})
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [44]:
@app.route('/admin/phnocheck',methods=["POST"])
def admin_phno_check():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        mobile = request.form['mobile']
        db_response = collection.find_one({"mobile": mobile})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [45]:
@app.route('/admin/aadharcheck',methods=["POST"])
def admin_aadhar_check():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        aadhar = request.form['aadhar']
        db_response = collection.find_one({"aadhar": aadhar})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [46]:
@app.route('/admin/login',methods=["POST"])
def admin_login():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['admin_id']
        passw = request.form['password']
#         passw = decryption(request.form['password'])
        db_response = collection.find_one({"admin_id":adminId})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == decryption(db_response['encrypted_pwd']):
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
            else:
                return json.dumps({"status": "failed","response": "You are already logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [47]:
@app.route('/super-admin/login',methods=["POST"])
def super_admin_login():
    db = myclient["accounts"]
    collection = db["super_admin"]
    if request.method == 'POST':
        email = request.form['email']
        passw = request.form['password']
#         passw = decryption(request.form['password'])
        db_response = collection.find_one({"email": email,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == db_response['password']:
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Password Reset

In [48]:
# STUDENT RESET PASSWORD
@app.route('/student/reset-password',methods=["POST"])
def student_reset_password():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        old_password = request.form['old_password']
        new_password = request.form['new_password']
        encrypted = encryption(new_password)
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            if old_password == db_response['password']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":new_password,"encrypted_pwd":encrypted}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [49]:
@app.route('/teacher/reset-password',methods=["POST"])
def teacher_reset_password():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        old_password = request.form['old_password']
        new_password = request.form['new_password']
        encrypted = encryption(new_password)
        db_response = collection.find_one({"regId": regId})
        if db_response != None:
            if old_password == db_response['password']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":new_password,"encrypted_pwd":encrypted}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "old password is incorrect"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [50]:
@app.route('/admin/reset-password',methods=["POST"])
def admin_reset_password():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        admin_id = request.form['admin_id']
        old_password = request.form['old_password']
        print(old_password)
        new_password = request.form['new_password']
        encrypted = encryption(new_password)
        db_response = collection.find_one({"admin_id": admin_id})
        if db_response != None:
            if old_password == db_response['password']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":new_password,"encrypted_pwd":encrypted}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "old password is incorrect"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Registration

In [51]:
@app.route('/student/registration',methods=["POST"])
def student_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["student"]
    data = {}
    dataAccount = {}
    response_string = ""

        
    if request.method == 'POST':
        dataAccount = {
                "name": request.form['name'],
                "regno": request.form['regno'],
                "password": request.form['password'],
                "encrypted_pwd": encryption(request.form['password']),
                "aadhar": request.form['aadhar'],
                "email": request.form['email'],
                "schoolname": request.form['schoolname'],
                "loggedIn": False,
                "coursePointer": "none",
                "subjectPointer": "none",
                "chapterPointer": "none",
                "raise_for_help": "no",
                "question_paper_received":"False",
                "teacherPointer": None,
                "Exam Attending" : None,
                "exam status": "None",
                "dob": request.form['dob'],          # MM-DD-YYYY
                "sex": request.form['sex'],
                "mobile": request.form['mobile'],
                "rollNumber": "none",  # public exam number
                "address": request.form['address'],  # string
                "pincode": request.form['pincode'],  #int
                "class": request.form['class'],
                "section":request.form["section"],
                 "isEnrolled": False,# int
                "testScore": {},
                "examScore": {},
                "enrolledCourses": {},
                "lastAccessed": "none",
                "extratime":""
            }
        
        db_check_user = collection.find_one(
            {
                "regno":dataAccount["regno"],
                "name":dataAccount["name"]
            }
        ) #check if user account exists in student collection
        
        if not(db_check_user):
            db_response = collection.insert_one(dataAccount)
#             data = {
# #                 "parentId": db_response.inserted_id,
                
#             }
            
#             db_response_account = collection_student.insert_one(data) #insert in students_profile collection
            
            if db_response.acknowledged == True:
                print(dataAccount["class"])
                if int(dataAccount["class"]) <= 10:
                    db = myclient['Class']
                    collection = db['uptoSSLC']
                    course_response = collection.find_one({},{"_id":0})
                    db = myclient["accounts"]
                    collection = db["student"]
                    
                    update_result = collection.update_one({"regno":dataAccount["regno"]},{"$set":{"enrolledCourses":course_response["subjects"]}})
                    if update_result.acknowledged == True:
                        response_string = {"status": "success","response":"ok"}
                    else:
                        response_string = {"status":"failed","response":"Database error"}
                else:
                    response_string = {"status": "success","response":"ok"}
                            
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

In [52]:
@app.route('/teacher/registration',methods=["POST"])
def teacher_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["teacher"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        
        data = {
            "name": request.form['name'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "regId": request.form['regId'],
            "schoolname":request.form['schoolname'],
            "address":request.form['address'],
            "mobile": request.form['mobile'],
            "email": request.form['email'],
            "password": request.form['password'],  
            "isEnrolled": False,
            "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "enrolledCourses": {"9":{},"10":{},"11":{},"12":{}},
            "coursePointer": "none",
            "createdTests": {},
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        print(data)
        
        db_check_user = collection.find_one({"regId":data["regId"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

In [53]:
@app.route('/admin/registration',methods=["POST"])
def admin_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["admin"]
    data = {}
    response_string = {}
    
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "admin_id": request.form['admin_id'], 
            "schoolname": request.form['schoolname'],
            "aadhar": request.form['aadhar'], 
            "mobile": request.form['mobile'],
            "email": request.form['email'],
            "password": request.form['password'],  
            "encrypted_pwd": encryptor.encrypt(bytes(request.form['password'],'utf-8')),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0],                        #YYYY-MM-DD
            "loggedIn": False,
            "address": request.form['address'],
            "pincode": request.form['pincode']
        }
        
        db_check_user = collection.find_one({"admin_id":data["admin_id"]}) #check if user account exists
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

In [54]:
@app.route('/super-admin/registration',methods=["POST"])
def super_admin_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["super_admin"]
    data = {}
    response_string = {}
    
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "dob": request.form['dob'],
            "mobile": request.form['mobile'],
            "sex": request.form['sex'],
            "email": request.form['email'], 
            "password": request.form['password'],  
            "encrypted_pwd": encryptor.encrypt(bytes(request.form['password'],'utf-8')),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0],                        #YYYY-MM-DD
            "loggedIn": False,
        }
        
        db_check_user = collection.find_one({"name":data["name"],"email":data["email"]}) #check if user account exists
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# Question-Retrival

In [55]:
@app.route('/question',methods=["POST"])
def question():
    db=myclient["English-I-QuestionBank"]
    collection=db["EnglishQuestionPaper"]
    if request.method=='POST':
        db_response=collection.find_one()
        ids=db_response.get('_id')
        regno=request.form['regno']
        db_student=myclient["accounts"]
        collection_student=db_student["student"]
        db_response_student=collection_student.update_one({"regno":regno},{ "$set": { "Exam Attending": ids,"question_paper_received": "True" } })
        if db_response != None:
            return json_util.dumps(db_response)
    return json.dumps({"status": "failed","response": "Question not found"})


In [56]:
@app.route('/question2',methods=["POST"])
def question2():
    db=myclient["English-II-QuestionBank"]
    collection=db["English-II-Paper"]
    if request.method=='POST':
        db_response=collection.find_one()
        ids=db_response.get('_id')
        regno=request.form['regno']
        db_student=myclient["accounts"]
        collection_student=db_student["student"]
        db_response_student=collection_student.update_one({"regno":regno},{ "$set": { "Exam Attending": ids,"question_paper_received": "True" } })
        if db_response != None:
            return json_util.dumps(db_response)
    return json.dumps({"status": "failed","response": "Question not found"})


# REVIEW ANSWERS

In [57]:
'''
GET:
    parameter: 
        regno->str
    return value:
        user_document->dict
        
'''
@app.route('/examination/response/review', methods=['GET', 'POST'])
def review():
    if request.method == 'GET':
        regno = request.args.get('regno')
        collection = myclient["examination"]["formattedResponse"]
        user_document = collection.find_one({'regno':regno},{'_id':0})
        if user_document != None:
            return make_response(json.dumps({'regno': regno, 'answer':user_document}), 200)
        else:
            return make_response(json.dumps({'regno': regno, 'message':'User document not found or processing in progress'}), 404)
    
    return make_response(json.dumps({'message':'Forbidden Access'}), 403)

# PRINTING

In [58]:
@app.route('/printpdf',methods=["POST"])
def print_pdf():
    db=myclient["English-I-QuestionBank"]
    collection=db["EnglishQuestionPaper"]
    if request.method=='POST':
        db_response=collection.find_one({},{"_id":0})
     
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "failed","response": "Question not found"})

# Question-Setting-APIs

In [59]:
@app.route('/insertSynonyms',methods=["POST"])
def insert_questions():
    db = myclient["English-I-QuestionBank"]
    collection = db["Synonyms"]

    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Synonyms"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Synonyms"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [60]:
@app.route('/insertSynonymsWithoutOptions',methods=["POST"])
def insert_synonyms_without_options_questions():
    db = myclient["English-I-QuestionBank"]
    collection = db["Synonyms without options"]

    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Synonyms without options"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Synonyms without options"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [61]:
@app.route('/insertAntonyms',methods=["POST"])
def insert_Antonyms():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["Antonyms"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Antonyms"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Antonyms"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [62]:
@app.route('/insertAntonymsWithoutOptions',methods=["POST"])
def insert_antonyms_without_options_questions():
    db = myclient["English-I-QuestionBank"]
    collection = db["Antonyms without options"]

    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Antonyms without options"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Antonyms without options"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [63]:
@app.route('/insertAnsVocabulary',methods=["POST"])
def insert_AnsVocabulary():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["AnsVocabulary"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["AnsVocabulary"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["AnsVocabulary"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [64]:
@app.route('/insertAnsGrammar1',methods=["POST"])
def insert_AnsGrammar1():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["AnsGrammar1"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["AnsGrammar1"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["AnsGrammar1"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [65]:
@app.route('/insertAnsGrammar2',methods=["POST"])
def insert_AnsGrammar2():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["AnsGrammar2"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["AnsGrammar2"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["AnsGrammar2"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [66]:
@app.route('/insertPunctuation',methods=["POST"])
def insert_Punctuation():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["Punctuation"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Punctuation"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Punctuation"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [67]:
@app.route('/insertShortQuestions',methods=["POST"])
def insert_ShortQuestions():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["ShortQuestions"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["ShortQuestions"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["ShortQuestions"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [68]:
@app.route('/insertProseParaQuestions',methods=["POST"])
def insert_ProseParaQuestions():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["ProseParaQuestions"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["ProseParaQuestions"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["ProseParaQuestions"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [69]:
@app.route('/insertMemoryPoem',methods=["POST"])
def insert_MemoryPoem():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["MemoryPoem"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["MemoryPoem"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["MemoryPoem"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [70]:
@app.route('/insertPoeticLines',methods=["POST"])
def insert_PoeticLines():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["PoeticLines"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["PoeticLines"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["PoeticLines"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [71]:
@app.route('/insertFigureOfSpeech',methods=["POST"])
def insert_FigureOfSpeech():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["FigureOfSpeech"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["FigureOfSpeech"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["FigureOfSpeech"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [72]:
@app.route('/insertPoemParaQuestions',methods=["POST"])
def insert_PoemParaQuestions():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["PoemParaQuestions"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["PoemParaQuestions"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["PoemParaQuestions"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [73]:
@app.route('/insertComprehensionQuestions',methods=["POST"])
def insert_ComprehensionIQuestions():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["ComprehensionQuestions"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["ComprehensionQuestions"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["ComprehensionQuestions"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [74]:
@app.route('/insertErrorSpotting',methods=["POST"])
def insert_ErrorSpotting():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["ErrorSpotting"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["ErrorSpotting"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["ErrorSpotting"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [75]:
@app.route('/insertImageComprehensionQuestions',methods=["POST"])
def insert_ImageComprehensionQuestions():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["ImageComprehensionQuestions"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["ImageComprehensionQuestions"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["ImageComprehensionQuestions"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

# Whole Question Paper Insertion

In [76]:
@app.route('/insertEnglishIQuestionPaper',methods=["POST"])
def insert_English_I_QuestionPaper():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["EnglishQuestionPaper"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["EnglishQuestionPaper"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["EnglishQuestionPaper"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

# Playbook

In [77]:
@app.route('/insertPlaybook',methods=["POST"])
def insert_Playbook():
#   declaration space
    db = myclient["English-I-QuestionBank"]
    collection = db["Playbook"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user']})
    # db_response = collection.insert_one(data)

        if existing_data == None:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            db = myclient["English-I-QuestionBank-HistoryDB"]
            collection = db["Playbook"]
            history_response = collection.insert_one(existing_data)
            if history_response.acknowledged == True:
                db = myclient["English-I-QuestionBank"]
                collection = db["Playbook"]
                collection.delete_one({'user': incoming_data['user']})
                collection.insert_one(incoming_data)
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

# Playbook - retrieval

In [78]:
@app.route('/playbook',methods=["POST"])
def playbook():
    db=myclient["English-I-QuestionBank"]
    collection=db["Playbook"]
    if request.method=='POST':
        db_response=collection.find_one()
        ids=db_response.get('_id')
        regno=request.form['regno']
        db_student=myclient["accounts"]
        collection_student=db_student["student"]
#         db_response_student=collection_student.update_one({"regno":regno})
        if db_response != None:
            return json_util.dumps(db_response)
    return json.dumps({"status": "failed","response": "Playbook not found"})


# FINAL ANSWER SUBMISSION

In [79]:
'''
    POST:
        parameter:
            user_document->dict
        return:
            status message from server (Response 200)
'''

@app.route('/examination/response/final-submission',methods=["POST", "GET"])
def final_submission():
    if request.method == "POST":
        answerCollection = myclient["examination"]["Answersheet"]
        user_document = request.get_json()
        user_document['tag'] = "final"
        answerCollection.insert_one(user_document)
        return make_response(json.dumps({"status": "success", "message":"Submission received"}),200)
    else:
        return make_response(json.dumps({"status": "failed", "message":"Forbidden Access"}),403)

# ANSWER - PRINTING

In [80]:
# @app.route('/answerprintpdf',methods=["POST"])
# def p_pdf():
#     db=myclient["examination"]
#     collection=db["Answersheet"]
#     if request.method=='POST':
#         db_response=collection.find_one({},{"_id":0})
#         if db_response != None:
#             return json.dumps(db_response)
#     return json.dumps({"status": "failed","response": "Answer Paper not found"})

In [81]:
@app.route('/answerprintpdf',methods=["POST"])
def p_pdf():
    db=myclient["examination"]
    collection=db["Answersheet"]
    if request.method=='POST':
        
        db_response=list(collection.find({},{"_id":0}))

        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "failed","response": "Answer Paper not found"})

In [82]:
%run -i book-guide-english10.py

ValueError: source code string cannot contain null bytes

In [83]:
@app.route('/uploads', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        ch_no = request.form['ch_no']
        unit_type = request.form['unit_type']
        heading  = request.form['heading']
        f = request.files['file']
        filename = f.filename
        
        if filename != '' and ch_no != '' and unit_type != '' and heading != '' :
            file_ext = os.path.splitext(filename)[1]
            if file_ext not in app.config['UPLOAD_EXTENSIONS']:
                return json.dumps({"status": "failed","response": "File Type not accepted"})
        else:
            return json.dumps({"status": "failed","response": "No file contents"})
        
        f.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        
        return json.dumps({"status": "success","response": "uploaded successfully", "data": parse_text(UPLOAD_FOLDER+'/'+filename,ch_no,unit_type,heading)})
    
    return json.dumps({"status": "failed","response": "Permission Denied"})

# LESSON CONTENT - RETRIEVAL

In [84]:
@app.route('/content/lesson',methods=["POST"])
def lesson_rec():
    db = myclient["contentsDB"]
    if request.method == 'POST':
        ch_no = request.form['chapter_no']
        ch_no = int(ch_no)
        unit_type = request.form['type']
    
        if(unit_type == 'poem'):
            collection = db["poems"]
            db_response = collection.find_one({"chapter no":ch_no},{"_id":0})
            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})

        elif(unit_type == 'prose'):

            collection = db["proses"]

            db_response = collection.find_one({"chapter no":ch_no},{"_id":0})
            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})

        elif(unit_type == 'supplementary'):

            collection = db["supplementaries"]

            db_response = collection.find_one({"chapter no": ch_no},{"_id":0})

            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})
        else:
            return json.dumps({"status": "failed","response": "Unit types not matched"})
    else:
        return json.dumps({"status": "failed","response": "post method error"})
    

# SOLUTIONS GUIDE - RETRIEVAL

In [85]:
@app.route('/content/solutions-guide',methods=["POST"])
def solution_rec():
    db = myclient["contentsDB"]
    if request.method == 'POST':
        ch_no = request.form['chapter_no']
        ch_no = int(ch_no)
        unit_type = request.form['type']
    
        if(unit_type == 'poem'):
            collection = db["poem-guide"]
            db_response = collection.find_one({"chapter no":ch_no},{"_id":0})
            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})

        elif(unit_type == 'prose'):

            collection = db["prose-guide"]

            db_response = collection.find_one({"chapter no":ch_no},{"_id":0})
            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})

        elif(unit_type == 'supplementary'):

            collection = db["supplementary-guide"]

            db_response = collection.find_one({"chapter no": ch_no},{"_id":0})

            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})
            
        elif(unit_type == 'grammar'):

            collection = db["grammar"]

            db_response = collection.find_one({"chapter no": ch_no},{"_id":0})

            if db_response != None:
                res=json.dumps({"status": "success","response": db_response})
                return res
            else:
                return json.dumps({"status": "failed","response": "Chapter not found"})

        else:
            return json.dumps({"status": "failed","response": "Unit types not matched"})
    else:
        return json.dumps({"status": "failed","response": "post method error"})
    

In [86]:
import socket 
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)

In [87]:
l=set()
def add(ip):
    l.add(ip)

In [88]:
@app.route("/get",methods=["GET"])
def get_my_ip():
    return str(l)

In [89]:
@app.route('/proctor/teacher-registration',methods=["POST"])
def teacher_Reg():
#   declaration space
    db = myclient["accounts-db"]
    collection = db["teacher"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        data = {
            "regId": request.form['regId'],
            "name": request.form['name'],
            "aadhar": request.form['aadhar'],
            "email": request.form['email'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],
            "encrypted_pwd": encryption(request.form['password']),
            "loggedIn": False
        }
        
        db_check_user = collection.find_one({"regId":data["regId"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

In [90]:
@app.route('/proctor/login',methods=["POST"])
def tea_login():
    db = myclient["accounts-db"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == decryption(db_response['encrypted_pwd']):
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                        
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
                
            elif db_response['loggedIn'] == True:
                response_string = {"status": "successful","response":"ok"}
                return json.dumps(response_string, indent=4)
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [91]:
@app.route('/proctor/logout',methods=["POST"])
def tea_logout():
    db = myclient["accounts-db"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are not logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [92]:
@app.route('/proctor/reset-password',methods=["POST"])
def tea_reset_password():
    db = myclient["accounts-db"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        aadhar = request.form['aadhar']
        password = request.form['password']
        db_response = collection.find_one({"regId": regId})
        if db_response != None:
            if aadhar == db_response['aadhar']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":password}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# EXAMINATION -STATUS CHANGE

In [93]:
@app.route('/examination/status',methods=["POST"])
def status_change():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        status = request.form['exam status']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:

            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"exam status":status}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
           
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# EXAMINATION - RAISE FOR HELP

In [94]:
@app.route('/examination/raiseforhelp',methods=["POST"])
def raise_for_help():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        r_help = request.form['raise_for_help']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:

            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"raise_for_help":r_help}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
           
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [95]:
@app.route('/examination/extratime',methods=["POST"])
def extra_time():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        e_time = request.form['extratime']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:

            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"extratime":e_time}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
           
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [96]:
@app.route('/examination/gettime',methods=["POST"])
def get_time():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            return json.dumps({"status":"success","response":db_response["extratime"]})
           
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# ENROLLMENT

In [97]:
@app.route('/student/enrollcheck',methods=["POST"])
def enroll_check():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = collection.find_one({"regno": regno})
        if db_response != None:
            
            return json.dumps({"status":"success","response":db_response["isEnrolled"]})
           
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# EXAMINATION - LIVE PANEL 1

In [98]:
@app.route('/examination/panel',methods=["GET", "POST"])
def status():
    
    start = time.time()
    
    final_response = {}
    final_response["status"] = "success"
    try:
        # call the server_info() to verify that client instance is valid
        myclient.server_info() # will throw an exception
    except:
        return json.dumps({"status": "failed"})
    finally:
        final_response["time lapsed"] = time.time() - start
    
    db = myclient["English-I-QuestionBank"]
    collection = db["EnglishQuestionPaper"]
    db_response = collection.find_one()
#     response = collection.find_one({'_id': ObjectId(request.form['id'])})
    response = collection.find_one({'_id': db_response.get('_id')})
    final_response["date of exam"] = response["date of exam"]

    return json.dumps(final_response)

#     return json.dumps({"status": "success", "time lapsed": time.time() - start})


# EXAMINATION - LIVE PANEL 2

In [99]:
@app.route('/examination/livedata',methods=["POST"])
def live():
    if request.method=='POST':
        db = myclient["English-I-QuestionBank"]
        collection = db["EnglishQuestionPaper"]
        db_response=collection.find_one()
        
        ids=db_response.get('_id')
        
        result_list = []
        
        db_student=myclient["accounts"]
        collection_student=db_student["student"]
        
        db_response_student=collection_student.find({ "Exam Attending": ids },{"password":0,"aadhar":0,"coursePointer":0,"subjectPointer":0,"chapterPointer":0,"teacherPointer":0,"_id":0,"created":0,"Exam Attending":0, "last login":0})
       
        
        
        if db_response_student != None:
            response_db = myclient["examination"]
            answers_collection = response_db["Answersheet"]
            db_response_student = list(db_response_student)
            user_answer = answers_collection.find()

            
            regno_list = []
            for student in db_response_student:
                regno_list.append(student['regno'])
                
            for answer in user_answer:
                if answer['regno'] in regno_list:
#                     db_response_student[regno_list.index(student['regno'])]['exam status'] = student['exam status']
                    db_response_student[regno_list.index(answer['regno'])]['submitted'] = answer['submitted']
#                     db_response_student[regno_list.index(answer['regno'])]['start time'] = answer['start time']   //Time at which the user submits first part of answer document
                
            return json_util.dumps(db_response_student)
    
    return json.dumps({"status": "failed"})

# SEARCH

In [100]:
'''
POST:
    description:
        It searches for the word in the user document.
    database:
        name : examination
        collection : rawResponse
    parameters:
        regno : string
    return:
        200:
            Path of the word searched.
        404:
            (1). Document not found.
            (2). Word doesn't exist in the document
'''

@app.route('/examination/search', methods = ['GET', 'POST'])
def search():
    
    answersCollection = myclient["examination"]["rawResponse"]
    
    if request.method == 'POST':
        regno = request.json['regno']
        search_word = request.json['search word']
        answer_document = answersCollection.find_one({'regno': regno}, {'_id':0, 'timestamp':0, 'regno':0,"question paper id":0,"title":0})
    
        if answer_document is None:
            return make_response(json.dumps({"status": "failed", "message": "No answer found for user with regno {r}".format(r=regno)}), 404)
        else:
            matches = []
            for section in answer_document:
                for part in answer_document[section]:
                    for question in answer_document[section][part]:
                        
                        answer_array = answer_document[section][part][question]["answer"]  # the answer array for the user
                        answer_array = [ans.lower() for ans in answer_array]               # Converting to lower case for better matching
                    
                        if len(answer_array) == 1:
                            try:
                                idx = answer_array[0].index(search_word.lower())
                                matches.append({"section": section, "part": part, "question": question, "answer index": idx})
                            except:
                                pass
                        else:
                            if search_word.lower() in answer_array:
                                matches.append({"section": section, "part": part, "question": question, "answer index": answer_array.index(search_word.lower())})

            if len(matches) == 0:
                return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document with regno {r}".format(r=regno)}), 200)
            else:
                return make_response(json.dumps({"status": "success", "matches": matches}), 200)

# QUESTION - SEARCH

In [101]:
# '''
# POST:
#     description:
#         It searches for the word in the user document.
#     database:
#         name : examination
#         collection : rawResponse
#     parameters:
#         regno : string
#     return:
#         200:
#             Path of the word searched.
#         404:
#             (1). Document not found.
#             (2). Word doesn't exist in the document
# '''

# @app.route('/examination/question/search', methods = ['GET', 'POST'])
# def ques_search():
    
#     quesCollection = myclient["English-I-QuestionBank"]["EnglishQuestionPaper"]
    
#     if request.method == 'POST':
#         search_word = request.json['search word']
#         doc = quesCollection.find_one({}, {'_id':0,"type of test":0,"school name":0,"standard":0,"title":0,"date of exam":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})
         
    
#         if doc is None:
#             return make_response(json.dumps({"status": "failed", "message": "No question found"}), 404)
#         else:
#             occurences = []
# #             search_word = "Read the passage given below and answer the questions that follow"

#             for section in doc:
#                 parts = doc[section]
#                 if "title" in parts:
#                     del parts["title"]
#                 if "marks" in parts:
#                     del parts["marks"]
#                 if "heading" in parts:
#                     del parts["heading"]
#                 for part in parts:
                    
#                     if "marks" in parts[part]:
#                         del parts[part]["marks"]
#                     if "heading" in parts[part]:
#                         del parts[part]["heading"]
#                     for ques in parts[part]:
#                         ques_data = parts[part][ques]
#                         if ques_data != None:
#                             if "paragraph" in ques_data.keys() and ques_data["paragraph"]!=None:
#                                 if ques_data["paragraph"][0].find(search_word)!=-1:
#                                     occurences.append({"section": section, "part": part, "question": ques})
#                             if "heading" in ques_data.keys() and ques_data["heading"]!=None:
#                                 if ques_data["heading"].find(search_word)!=-1:
#                                     occurences.append({"section": section, "part": part, "question": ques})

#                             if "question" in ques_data.keys() and ques_data["question"]!=None:
                                
#                                 if type(ques_data["question"]) == dict:
#                                     for key in ques_data["question"]:
#                                         if ques_data["question"][key].find(search_word)!=-1:
#                                             occurences.append({"section": section, "part": part, "question": ques, "question key": key})
#                                 elif ques_data["question"].find(search_word)!=-1:
#                                     occurences.append({"section": section, "part": part, "question": ques})
            
#             if len(occurences) == 0:
#                 return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
#             else:
#                 return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

            

In [102]:
'''
POST:
    description:
        It searches for the word in the user document.
    database:
        name : examination
        collection : rawResponse
    parameters:
        regno : string
    return:
        200:
            Path of the word searched.
        404:
            (1). Document not found.
            (2). Word doesn't exist in the document
'''


@app.route('/examination/question/search', methods = ['GET', 'POST'])
def ques_search():
    
    quesCollection = myclient["English-I-QuestionBank"]["EnglishQuestionPaper"]
    
    if request.method == 'POST':
        
         
        data = request.get_json()
        search_word = data['search word']
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno=data['regno']
        
        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
            doc = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype},{'_id':0,"school name":0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})

        else:
            doc = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname},{'_id':0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})

        if doc is None:

            return make_response(json.dumps({"status": "failed", "message": "No question found"}), 404)

        else:
            occurences = []

            del doc["date of exam"]
            del doc["standard"]
            del doc["type of test"]
            if "school name" in doc:
                del doc["school name"]

            for section in doc:
                parts = doc[section]
                if "title" in parts:
                    del parts["title"]
                if "marks" in parts:
                    del parts["marks"]
                if "heading" in parts:
                    del parts["heading"]
                for part in parts:
                    if "marks" in parts[part]:
                        del parts[part]["marks"]
                    if "heading" in parts[part]:
                        del parts[part]["heading"]

                    for ques in parts[part]:
                        ques_data = parts[part][ques]

                        if ques_data != None:
                            if "paragraph" in ques_data.keys() and ques_data["paragraph"]!=None:
                                if ques_data["paragraph"][0].lower().find(search_word)!=-1:
                                    occurences.append({"section": section, "part": part, "question": ques})

                                elif ques_data["paragraph"][0].find(search_word)!=-1:
                                    occurences.append({"section": section, "part": part, "question": ques})

                            if "heading" in ques_data.keys() and ques_data["heading"]!=None:

                                if ques_data["heading"].lower().find(search_word)!=-1:
                                    occurences.append({"section": section, "part": part, "question": ques})

                                elif ques_data["heading"].find(search_word)!=-1:
                                    occurences.append({"section": section, "part": part, "question": ques})

                            if "question" in ques_data.keys():
                                if type(ques_data["question"]) == dict:
                                    for key in ques_data["question"]:
                                        if type(ques_data["question"][key]) == dict:
                                             if "question" in ques_data["question"][key].keys():
                                                if type(ques_data["question"][key]["question"]) == dict:  
                                                    if ques_data["question"][key]["question"].lower().find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques})
                                                    elif ques_data["question"][key]["question"].find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques})

                                                if type(ques_data["question"][key]["question"]) == str:  
                                                    if ques_data["question"][key]["question"].lower().find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})
                                                    elif ques_data["question"][key]["question"].find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})

                                        if type(ques_data["question"][key]) == str:
                                            if ques_data["question"][key].lower().find(search_word) != -1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

                                            elif ques_data["question"][key].find(search_word) != -1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})


                                if type(ques_data["question"]) == str:

                                        if ques_data["question"].lower().find(search_word)!=-1:
                                            occurences.append({"section": section, "part": part, "question": ques})
                                        elif ques_data["question"].find(search_word)!=-1:
                                            occurences.append({"section": section, "part": part, "question": ques})

                                if type(ques_data["question"]) == list:
                                        for index in range(len(ques_data["question"])):
                                            if ques_data["question"][index].lower().find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":(index+1)})

                                            elif ques_data["question"][index].find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":(index+1)})


                            if "questions" in ques_data.keys():
                                if type(ques_data["questions"]) == dict:
                                    for key in ques_data["questions"]:
                                        if type(ques_data["questions"][key]) == dict: 
                                            if "question" in ques_data["questions"][key].keys():
                                                if type(ques_data["questions"][key]["question"]) == str:
                                                    if ques_data["questions"][key]["question"].lower().find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})
                                                    elif ques_data["questions"][key]["question"].find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

                                            if "heading" in ques_data["questions"][key].keys() and ques_data["questions"][key]["heading"] != None:
                                                if ques_data["questions"][key]["heading"].lower().find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})
                                                elif ques_data["questions"][key]["heading"].find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})

                                        if type(ques_data["questions"][key]) == str:
                                            if ques_data["questions"][key].lower().find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})
                                            if ques_data["questions"][key].find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

            if len(occurences) == 0:
                return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
            else:
                return make_response(json.dumps({"status": "success", "matches": occurences}), 200)



# COURSE - ENROLLMENT

In [103]:
@app.route('/student/hscgroup',methods=["POST"])
def student_hsc():
    db = myclient["Class"]
    collection = db["HSC"] 
    if request.method == 'POST':
        group = request.form['subjects']
        db_res = collection.find_one({},{"_id":0})
        if db_res != None:
            return json.dumps({group+"":db_res["groups"][group]})
    else:
        return json.dumps({"status": "failed","response": "User not found"})

In [104]:
@app.route('/student/hsc',methods=["POST"])
def student_hsc_grp():
    db = myclient["Class"]
    collection = db["HSC"]
        
    if request.method == 'POST':
        group = request.form["subjects"]
        db_res = collection.find_one({},{"_id":0})
        
        if db_res != None:
            return json.dumps({group+"":db_res["groups"][group]})
        
    else:
        return json.dumps({"status": "failed","response": "User not found"})

In [105]:
@app.route('/student/sslc',methods=["POST"])
def student_sslc():
    db = myclient["Class"]
    collection = db["SSLC"]
        
    if request.method == 'POST':
        db_res = collection.find_one({},{"_id":0})
        
        if db_res != None:
            return json.dumps({group+"":db_res["subjects"]})
        
    else:
        return json.dumps({"status": "failed","response": "User not found"})

# COURSE ENROLLMENT CONFIRMATION

In [106]:
@app.route('/student/confirm-course',methods=["POST"])
def student_confirm_course():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
#         course = request.json('regno');

        course = request.get_json()
        regno = course["regno"]
        std = course["class"]
        section = course["section"]
        subjects = course["groups"]
        db_response = collection.find_one({"regno":regno,"class":std,"section":section},{"_id":0})
        if db_response != None:
            if db_response["isEnrolled"] != True:
                update_result = collection.update_one({"regno":regno},{"$set":{"enrolledCourses":{"groups":subjects},"isEnrolled":True}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                else:
                    return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status":"failed","response":"User already enrolled"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})


# EXAMINER- REGISTRATION

In [107]:
@app.route('/examiner/registration',methods=["POST"])
def examiner_registration():
#   declaration space
    db = myclient["accounts"]
    collection = db["examiner"]
    data = {}
    response_string = {}
    if request.method == 'POST':
        data = {
            "name": request.form['name'],
            "hallno":request.form['hallno'],
            "aadhar": request.form['aadhar'],
            "dob": request.form['dob'],
            "sex": request.form['sex'],
            "email":request.form['email'],
            "address": request.form['address'],
            "pincode": request.form['pincode'],
            "examiner_id": request.form['examiner_id'],
            "schoolname":request.form['schoolname'],
            "mobile": request.form['mobile'],
            "password": request.form['password'],
            "isRegistered":False,
            "encrypted_pwd": encryption(request.form['password']),    
            "createdAt": str(datetime.datetime.now()).split(" ")[0], #YYYY-MM-DD
            "loggedIn": False,
            "pincode": request.form['pincode'] 
        }
        
        db_check_user = collection.find_one({"examiner_id":data["examiner_id"]}) #check if user account exists
        
        if not(db_check_user):
            db_response = collection.insert_one(data)
            
            if db_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        
        else:
            response_string = {"status": "failed","response": "User account already exists"}
        
    return json.dumps(response_string)

# EXAMINER - LOGIN

In [108]:
@app.route('/admin-portal/examiner-login',methods=["POST"])
def admin_portal_examiner_login():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examiner_id = request.form['examiner_id']
        passw = request.form['password']
        db_response = collection.find_one({"examiner_id": examiner_id,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != True:
                if passw == decryption(db_response['encrypted_pwd']):
                    update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": False},{"$set":{"loggedIn":True}})
                    if update_result.acknowledged == True and update_result.modified_count == 1:
                        response_string = {"status": "success","response":"ok"}
                        
                    else:
                        response_string = {"status":"failed","response":"database error"}
                    return json.dumps(response_string, indent=4)
                else:
                    return json.dumps({"status": "failed","response":"wrong password"})
                
            elif db_response['loggedIn'] == True:
                response_string = {"status": "successful","response":"You are already logged in on another device"}
                return json.dumps(response_string, indent=4)
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# EXAMINER-RESET-PASSWORD

In [109]:
@app.route('/admin-portal/examiner-reset-password',methods=["POST"])
def admin_portal_examiner_reset_password():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        aadhar = request.form['aadhar']
        password = request.form['password']
        db_response = collection.find_one({"examiner_id": examinerId})
        if db_response != None:
            if aadhar == db_response['aadhar']:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":password}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})
            else:
                return json.dumps({"status": "failed","response": "aadhar didn't match"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# EXAMINER-LOGOUT

In [110]:
@app.route('/admin-portal/examiner-logout',methods=["POST"])
def admin_portal_examiner_logout():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examinerId = request.form['examiner_id']
        passw = request.form['password']
        db_response = collection.find_one({"examiner_id": examinerId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Admin Portal - delete individual student

In [111]:
@app.route('/student/delete',methods=["POST"])
def student_delete():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        
        regno = request.form['regno']
        
        db_response = collection.find_one({"regno": regno})
   
        if db_response != None:
            
            delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

            return json.dumps({"status": "success", "response": "Student profile deletd for the regno "+regno}) 
        
        else:

            response_string = {"status": "failed","response": "User not found"}
            return json.dumps(response_string)
                             

# Admin Portal - delete all student based on class and section

In [112]:
@app.route('/student/delete_All',methods=["POST"])
def student_delete_All():
    db = myclient["accounts"]
    
    collection = db["student"]
    
    if request.method == 'POST':
        
        std = request.form["class"]
        
        sec = request.form["section"]
        
        db_response = collection.find({"class":std,"section":sec},{"_id":0})
        
        teachers_doc = []
            
        for doc in db_response:
            teachers_doc.append(doc)
            
        
        if teachers_doc != []:
            
                delete_students = collection.delete_many({"class":std,"section":sec})
   

                response_string = {"status": "success","response":"All students Records are deleted "}
                    
                
                return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Admin portal - delete individual teacher

In [113]:
@app.route('/teacher/delete',methods=["POST"])
def teacher_delete():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        
        regId = request.form['regId']
        schoolname = request.form['schoolname']
        
        db_response = collection.find_one({"regId": regId,"schoolname":schoolname})

        
        if db_response != None:
            
            delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

            return json.dumps({"status": "success", "response": "Teacher profile deletd for the regno "+regId}) 
            

        else:

            response_string = {"status": "failed","response": "User not found"}
                             
      

# Admin portal - delete all teachers

In [114]:
@app.route('/teacher/deleteAll',methods=["POST"])
def teacher_deleteAll():
    db = myclient["accounts"]
    
    collection = db["teacher"]
    
    if request.method == 'POST':
        schoolname = request.form['schoolname']
        
        db_response = collection.find({"schoolname":schoolname})
        
#         teachers_doc = []
            
#         for doc in db_response:
#             teachers_doc.append(doc)
            
        
#         if teachers_doc != []:
            
#                 delete_teachers = collection.delete_many({})

        if db_response != None:
            result = collection.delete_many({"schoolname":schoolname})
            if result.acknowledged == True:
                response_string = {"status": "success","response":"Teacher Records are deleted "}
            else:
                response_string = {"status": "failed","response":"database error"}
            return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Delete individual examiner

In [115]:
@app.route('/examiner/delete',methods=["POST"])
def examiner_delete():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        schoolname = request.form['schoolname']
        examiner_id = request.form['examiner_id']
        
        db_response = collection.find_one({"examiner_id": examiner_id,"schoolname":schoolname})
   
        if db_response != None:
            
            delete_student = collection.delete_one({"_id": ObjectId(db_response['_id'])})

            return json.dumps({"status": "success", "response": "Student profile deletd for the regno "+examiner_id}) 
        
        else:

            response_string = {"status": "failed","response": "User not found"}
            return json.dumps(response_string)
                             

In [116]:
@app.route('/examiner/deleteAll',methods=["POST"])
def examiner_deleteAll():
    db = myclient["accounts"]
    
    collection = db["examiner"]
    
    if request.method == 'POST':
        
        schoolname = request.form['schoolname']
        
        db_response = collection.find({"schoolname":schoolname})
        
#         teachers_doc = []
            
#         for doc in db_response:
#             teachers_doc.append(doc)
            
        
#         if teachers_doc != []:
            
#                 delete_teachers = collection.delete_many({})

        if db_response != None:
            result = collection.delete_many({"schoolname":schoolname})
            if result.acknowledged == True:
                response_string = {"status": "success","response":"Examiner Records are deleted "}
            else:
                response_string = {"status": "failed","response":"database error"}
            return json.dumps(response_string, indent=4)
          
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# Student panel

In [117]:
@app.route('/student/getAll',methods=["GET", "POST"])
def student_get():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":True},{"_id":0,"Exam Attending":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [118]:
@app.route('/student/admin-approval',methods=["GET", "POST"])
def student_admin_approval():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":False},{"_id":0,"Exam Attending":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [119]:
@app.route('/student/approved',methods=["GET", "POST"])
def student_admin_approved():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']

        db_response = collection.find_one({"regno":regno},{"_id":0,"encrypted_pwd":0})
        if db_response != None:
            update_result = collection.update_one({"regno":regno},{"$set":{"isEnrolled":True}})
            if update_result.acknowledged == True:
                db_response = collection.find_one({"regno":regno},{"_id":0,"encrypted_pwd":0})
                print(db_response)
                return json.dumps({"status": "success","response": "ok"})
        return json.dumps({"status": "user not found","response": "User not found"})

# Teacher panel

In [120]:
@app.route('/teacher/getAll',methods=["GET", "POST"])
def teacher_get():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":True},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [121]:
@app.route('/teacher/admin-approval',methods=["GET", "POST"])
def teacher_admin_approval():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        db_response = list(collection.find({"isEnrolled":False},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [122]:
@app.route('/teacher/approved',methods=["GET", "POST"])
def teacher_admin_approved():
    db = myclient["accounts"]
    collection = db["teacher"]
    regId = request.form['regId']
    if request.method == 'POST':
        db_response = collection.find_one({"regId":regId},{"_id":0,"encrypted_pwd":0})
        if db_response != None:
            update_result = collection.update_one({"regId":regId},{"$set":{"isEnrolled":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Examiner panel

In [123]:
@app.route('/examiner/getAll',methods=["GET", "POST"])
def examiner_get():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [124]:
@app.route('/examiner/admin-approval',methods=["GET", "POST"])
def examiner_admin_approval():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        db_response = list(collection.find({"isRegistered":False},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [125]:
@app.route('/examiner/approved',methods=["GET", "POST"])
def examiner_admin_approved():
    db = myclient["accounts"]
    collection = db["examiner"]
    examiner_id = request.form['examiner_id']
    if request.method == 'POST':
        db_response = collection.find_one({"examiner_id":examiner_id},{"_id":0,"encrypted_pwd":0})
        if db_response != None:
            update_result = collection.update_one({"examiner_id":examiner_id},{"$set":{"isRegistered":True}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# ADMIN SIDE- Student course enrollment Web

In [126]:
@app.route('/student/enroll-course',methods=["POST"])
def student_enroll_course():
    db = myclient["Class"]
    collection = db["HSC"]
    sub = {}
    if request.method == 'POST':
        regno = request.form["regno"]
        std = request.form["std"]
        section = request.form["section"]
        group = request.form["group"]
        schoolname = request.form["schoolname"]
        
        db_res = collection.find_one({},{"_id":0})

        if db_res != None:
            sub = {group+"":db_res["groups"][group]}
            grp = {};
            grp["groups"] = sub
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"regno":regno,"class":std,"section":section,"schoolname":schoolname})
            if db_response != None:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":grp,"isEnrolled":True}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})

In [127]:
@app.route('/student/sslc-enroll-course',methods=["POST"])
def student_sslc_enroll_course():
    db = myclient["Class"]
    collection = db["SSLC"]
    sub = {}
    if request.method == 'POST':
        regno = request.form["regno"]
        
        db_res = collection.find_one({},{"_id":0})

        if db_res != None:
            sub = db_res["subjects"]
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"regno":regno})
            if db_response != None:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":sub,"isEnrolled":True}})
                if update_result.acknowledged:
                    return json.dumps({"status": "success","response": "ok"})
                return json.dumps({"status": "failed","response": "database error"})

# ADMIN AUTHENTICATION

In [128]:
@app.route('/admin-portal/admin-logout',methods=["POST"])
def admin_portal_admin_logout():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        passw = request.form['password']
        db_response = collection.find_one({"admin_id": adminId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
                update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == False and update_result.modified_count == 1:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

# Teacher records - deleteAll

In [129]:
@app.route('/admin-portal/teacher/deleteAll',methods=["POST"])
def admin_portal_teacher_deleteAll():
    db = myclient["accounts"]
   
    collection = db["teacher"]
   
    if request.method == 'POST':
       
        db_response = collection.find({})
       
        teachers_doc = []
           
        for doc in db_response:
            teachers_doc.append(doc)
           
       
        if teachers_doc != []:
           
                delete_teachers = collection.delete_many({})
   

                response_string = {"status": "success","response":"Teacher Records are deleted "}
                   
               
                return json.dumps(response_string, indent=4)
         
        else:
                return json.dumps({"status": "failed","response": "User not found"})

# ADMIN STUDENT PROFILE

In [130]:
@app.route('/admin-portal/student-profile',methods=["GET", "POST"])
def admin_portal_view_student_profile():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = (collection.find_one({"regno":regno},{"_id":0,"Exam Attending":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [131]:
@app.route('/admin-portal/edit-student-profile',methods=["GET", "POST"])
def admin_portal_student_edit_profile():
    db = myclient["accounts"]
    collection = db["student"]
    if request.method == 'POST':
        regno = request.form['regno']
        db_response = (collection.find_one({"regno":regno},{"_id":0,"Exam Attending":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [132]:
@app.route('/admin-portal/update-student-profile',methods=["GET", "POST"])
def admin_portal_student_update_profile():
    db = myclient["accounts"]
    collection = db["student"]
    admin_name = ''
    adminid = ''
    if request.method == 'POST':
        adminid = request.form['regno']
        
        admin_name = request.form['name']
        admin_password = request.form['password']
        encrypted_pwd = encryption(request.form['password'])
        admin_class =  request.form['class']
        admin_section = request.form['section']
        admin_aadhar =  request.form['aadhar']
        admin_mobile = request.form['mobile']
        admin_email = request.form['email']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']
        admin_address = request.form['address']
        admin_pincode = request.form['pincode']
        type(admin_name)
        db_response = (collection.find_one({"regno":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"class":admin_class,"section":admin_section,"password":admin_password,"email":admin_email,"encrypted_pwd":encrypted_pwd,"name":admin_name,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# ADMIN - TEACHER PROFILE

In [133]:
@app.route('/admin-portal/teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_view_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = (collection.find_one({"regId":regId},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

In [134]:
@app.route('/admin-portal/edit-teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_edit_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = (collection.find_one({"regId":regId},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [135]:
@app.route('/admin-portal/update-teacher-profile',methods=["GET", "POST"])
def admin_portal_teacher_update_profile():
    db = myclient["accounts"]
    collection = db["teacher"]
    admin_password = "";
    if request.method == 'POST':
        adminid = request.form['regId']
        admin_name = request.form['name']
        admin_password = request.form['password']
        encrypted_pwd = encryption(request.form['password'])
        admin_aadhar =  request.form['aadhar']
        admin_email =  request.form['email']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']
        admin_address = request.form['address']
        admin_pincode = request.form['pincode']
        db_response = (collection.find_one({"regId":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":admin_password,"encrypted_pwd":encrypted_pwd,"name":admin_name,"email":admin_email,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# Admin - Examiner - profile

In [136]:
@app.route('/admin-portal/examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_view_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examiner_id = request.form['examiner_id']
        db_response = (collection.find_one({"examiner_id":examiner_id},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

In [137]:
@app.route('/admin-portal/edit-examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_edit_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    if request.method == 'POST':
        examiner_id = request.form['examiner_id']
        db_response = (collection.find_one({"examiner_id":examiner_id},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [138]:
@app.route('/admin-portal/update-examiner-profile',methods=["GET", "POST"])
def admin_portal_examiner_update_profile():
    db = myclient["accounts"]
    collection = db["examiner"]
    admin_password = "";
    if request.method == 'POST':
        adminid = request.form['examiner_id']
        admin_name = request.form['name']
        admin_password = request.form['password']
        encrypted_pwd = encryption(request.form['password'])
        admin_aadhar =  request.form['aadhar']
        admin_email =  request.form['email']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_gender = request.form['gender']
        admin_address = request.form['address']
        admin_pincode = request.form['pincode']
        db_response = (collection.find_one({"examiner_id":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":admin_password,"encrypted_pwd":encrypted_pwd,"name":admin_name,"email":admin_email,"aadhar":admin_aadhar,"mobile":admin_mobile,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# ADMIN SIDE - TEACHER ENROLLMENT

In [139]:
@app.route('/admin-portal/teacher-course-enrollment',methods=["POST"])
def admin_portal_teacher_course_enrollment():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        courses = request.json["subjects"]
        regId = request.json["regId"]
        
#         regId = json.load(request.form['courses'])["regId"]
        db_response = collection.find_one({"regId": regId})
        print(regId)
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":courses}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# ADMIN PROFILE

In [140]:
@app.route('/admin-portal/admin-profile',methods=["GET", "POST"])
def admin_portal_admin_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        db_response = (collection.find_one({"admin_id":adminId},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [141]:
@app.route('/admin-portal/admin-edit-profile',methods=["GET", "POST"])
def admin_portal_admin_edit_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminId = request.form['adminId']
        db_response = (collection.find_one({"admin_id":adminId},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [142]:
@app.route('/admin-portal/admin-update-profile',methods=["GET", "POST"])
def admin_portal_admin_update_profile():
    db = myclient["accounts"]
    collection = db["admin"]
    if request.method == 'POST':
        adminid = request.form['admin_id']
        admin_name = request.form['name']
        admin_password = request.form['password']
        encrypted_pwd = encryption(request.form['password'])
        admin_aadhar =  request.form['aadhar']
        admin_mobile = request.form['mobile']
        admin_schoolname = request.form['schoolname']
        admin_dob =  request.form['dob']
        admin_email = request.form['email']
        admin_gender = request.form['gender']  
        admin_address = request.form['address']        
        admin_pincode = request.form['pincode']
        db_response = (collection.find_one({"admin_id":adminid}))
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"password":admin_password,"encrypted_pwd":encrypted_pwd,"name":admin_name,"aadhar":admin_aadhar,"mobile":admin_mobile,"email":admin_email,"dob":admin_dob,"sex":admin_gender,"schoolname":admin_schoolname,"address":admin_address,"pincode":admin_pincode}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
    return json.dumps({"status": "user not found","response": "User not found"})

# TIME TABLE

In [143]:
@app.route('/exam/time-table',methods=["POST"])
def insert_timetable():
    db = myclient["Class"]
    collection = db["exam - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname'],"exam-name": incoming_data['exam-name'],"class": incoming_data['class']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"] and incoming_data["exam-name"] == existing_data["exam-name"] and incoming_data["class"] == existing_data["class"]):
#                 collection.delete_many(existing_data)
#                 collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"already inserted"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)

In [144]:
@app.route('/delete/exam-time-table',methods=["POST"])
def insert_exam_timetable():
    db = myclient["Class"]
    collection = db["exam - time table"]
    response_string = {}
    if request.method == 'POST':
        admin_id = request.form['admin-id']
        std = request.form['std']
        schoolname = request.form['schoolname']
        exam_name = request.form['exam-name']
        existing_data = collection.find_one({"user": admin_id,"schoolname": schoolname,"exam-name": exam_name,"class": std},{"_id": 0})
       
        if existing_data != None:
            if(schoolname == existing_data["schoolname"] and exam_name == existing_data["exam-name"] and std == existing_data["class"]):
                collection.delete_many(existing_data)
                response_string = {"status": "success","response":"deleted"}
            else:
                response_string = {"status": "failed","response":"error"}

        else:
            response_string = {"status": "failed","response":"record not found"}
           
    return make_response(jsonify(response_string),200)

In [145]:
@app.route('/delete/class-time-table',methods=["POST"])
def delete_class_timetable():
    db = myclient["Class"]
    collection = db["student - time table"]
    response_string = {}
    if request.method == 'POST':
        admin_id = request.form['admin-id']
        std = request.form['std']
        section = request.form['section']
        schoolname = request.form['schoolname']
        existing_data = collection.find_one({"user": admin_id,"schoolname": schoolname,"section": section,"class": std},{"_id": 0})
       
        if existing_data != None:
            collection.delete_many(existing_data)
            response_string = {"status": "success","response":"deleted"}

        else:
            response_string = {"status": "failed","response":"record not found"}
           
    return make_response(jsonify(response_string),200)

In [146]:
@app.route('/delete/teacher-time-table',methods=["POST"])
def delete_teacher_timetable():
    db = myclient["Class"]
    collection = db["teacher - time table"]
    response_string = {}
    if request.method == 'POST':
        admin_id = request.form['admin-id']
        regId = request.form['regId']
        schoolname = request.form['schoolname']
        existing_data = collection.find_one({"user": admin_id,"regId":regId,"schoolname": schoolname},{"_id": 0})
       
        if existing_data != None:
            collection.delete_many(existing_data)
            response_string = {"status": "success","response":"deleted"}

        else:
            response_string = {"status": "failed","response":"record not found"}
           
    return make_response(jsonify(response_string),200)

In [147]:
@app.route('/update/exam-time-table',methods=["POST"])
def insert_update_exam_timetable():
    db = myclient["Class"]
    collection = db["exam - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        print(incoming_data)
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname'],"exam-name": incoming_data['exam-name'],"class": str(incoming_data['class'])},{"_id": 0})
        print(existing_data)
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"] and incoming_data["exam-name"] == existing_data["exam-name"] and incoming_data["class"] == existing_data["class"]):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"ok"}
            else:
                collection.delete_many(existing_data)
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            response_string = {"status": "failed","response":"database error"}
           
    return make_response(jsonify(response_string),200)

In [148]:
@app.route('/exam/get-timetable',methods=["GET", "POST"])
def student_timetable():
    db = myclient["Class"]
    collection = db["exam - time table"]
    if request.method == 'POST':
        std = request.form['class']
        schoolname = request.form['schoolname']
        db_response = (collection.find_one({"class":std,"schoolname":schoolname},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
        return json.dumps({"status": "failed","response":"Time table not found"})

In [149]:

@app.route('/question/get',methods=["POST"])
def question_get():
    
    
    if request.method=='POST':
        data = request.get_json()
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno=data['regno']
        

        db=myclient["English-I-QuestionBank"]
        collection=db["EnglishQuestionPaper"]
        answerCollection = myclient["examination"]["Answersheet"]

        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
            db_res=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"title":sub_name})
            ans_response = answerCollection.find_one({"regno":regno,"date of exam":exam_date,"standard":std,"type of test":testtype,"title":sub_name})

        else:
            db_res=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname,"title":sub_name})
            ans_response = answerCollection.find_one({"regno":regno,"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname,"title":sub_name})
            
        
        
        
        if ans_response == None:
        
            if db_res != None:
                ids=db_res.get('_id')
                db_student=myclient["accounts"]
                collection_student=db_student["student"]
                db_res_student=collection_student.find_one({"regno":regno})
                if db_res_student != None:
                    db_res_student=collection_student.update_one({"regno":regno},{ "$set": { "Exam Attending": ids,"question_paper_received": "True" } })

                    return json_util.dumps(db_res)
                else:
                    return json.dumps({"status":"failed","response":"student not found"})
            else:
                return json.dumps({"status": "failed","response": "Question not found"})
            
        else:
            return json.dumps({"status": "existed","response": "answer sheet already submitted"})




In [150]:
@app.route('/playbook/get',methods=["POST"])
def playbook_get():
    
#     db=myclient["English-II-QuestionBank"]
#     collection=db["English-II-Playbook"]
    if request.method=='POST':
        data = request.get_json()
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno=data['regno']
        
#         if "english" in sub_name.lower() and "II" in sub_name:
#             if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
#                 db_response=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype})
                
#             else:
#                 db_response=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname})
                
#             if db_response != None:
#                 db_student=myclient["accounts"]
#                 collection_student=db_student["student"]
#                 db_response_student=collection_student.find_one({"regno":regno})
#                 if db_response_student != None:
#                     return json_util.dumps(db_response)
#                 else:
#                     return json.dumps({"status":"failed","response":"student regno not found"})
#             else:
#                 return json.dumps({"status": "failed","response": "Playbook not found"})
                                  
#         else: #if "english" in lower(sub_name) and "I" in lower(sub_name):
        db=myclient["English-I-QuestionBank"]
        collection=db["Playbook"]

        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
            db_res=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype})

        else:
            db_res=collection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname})

        if db_res != None:
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_res_student=collection_student.find_one({"regno":regno})
            if db_res_student != None:
                return json_util.dumps(db_res)
            else:
                return json.dumps({"status":"failed","response":"student regno not found"})
        else:
            return json.dumps({"status": "failed","response": "Playbook not found"})




In [151]:
def processing_model(text):
#     p = Punctuator('E:/Model/Demo-Europarl-EN.pcl')
    text = p.punctuate(text)
        
#         #Gingerit
#     parser = GingerIt()
#     text = parser.parse(text)['result']

    input = text
    
    def combine_words(input,length):
        combined_inputs = []
        if len(splitted_input)>1:
            for i in range(len(input)-1):
                combined_inputs.append(input[i]+" "+last_word_of(splitted_input[i+1],length)) #add the last word of the right-neighbour (overlapping) sequence (before it has expanded), which is the next word in the original sentence
            return combined_inputs, length+1

    def remove_duplicates(input, length):
        bool_broke=False #this means we didn't find any duplicates here
        for i in range(len(input) - length):
            if input[i]==input[i + length]: #found a duplicate piece of sentence!
                for j in range(0,length): #remove the overlapping sequences in reverse order
                    del input[i + length - j]
                bool_broke = True
                break #break the for loop as the loop length does not matches the length of splitted_input anymore as we removed elements
        if bool_broke:
            return remove_duplicates(input, length) #if we found a duplicate, look for another duplicate of the same length
        return input

    def last_word_of(input,length):
        splitted = input.split(" ")
        if len(splitted)==0:
            return input
        else:
            return splitted[length-1]
        
    splitted_input = input.split(" ")
    word_length = 1
    splitted_input,word_length = combine_words(splitted_input,word_length)

    intermediate_output = False

    while len(splitted_input)>1:
        splitted_input = remove_duplicates(splitted_input,word_length) #look whether two sequences of length n (with distance n apart) are equal. If so, remove the n overlapping sequences
        splitted_input, word_length = combine_words(splitted_input,word_length) #make even bigger sequences
        if intermediate_output:
            print(splitted_input)
            print(word_length)
    output = splitted_input[0]

    print(output)
    return output
  
class Answer_Formatter:
    
    allowed_types = ["MCQCH", "MCQN", "COMP", "COMPIMG","TRAPIC","POEM","FORMAL","INFORMAL"]
    
    def process(self, answer):
        
        for index in range(len(answer)):
            if answer[index] !=None:
                if len(answer[index].split(" ")) == 1:
                    pass
                else:
                    answer[index] = processing_model(answer[index])

        return answer
    
    def format(self, document, is_section=False, is_part=False):
        if is_section:
            for part in document:
                for ques in document[part]:
                    if document[part][ques]["type"] in self.allowed_types: 
                        document[part][ques]["answer"] = self.process(document[part][ques]["answer"])
                    
        elif is_part:
            for ques in document:
                if document[ques]["type"] in self.allowed_types:
                    document[ques]["answer"] = self.process(document[ques]["answer"])
        
        return document
      
def format_function(sub_name,exam_date,testtype,std,schoolname,document, section, part, regno, is_section=False, is_part=False):
    
    print("Processing {regno} response in formatted Collection".format(regno=regno))
    
    formatted_document = Answer_Formatter().format(document, is_section, is_part)
    
        
    formattedCollection = myclient["examination"]["formattedResponse-paperI"]
    rawCollection = myclient["examination"]["rawResponse-paper-I"]

    user_document = formattedCollection.find_one({"regno": regno})

    if user_document == None:
        user_document = {}
        user_document['timestamp'] = datetime.datetime.now().strftime("%c")
        user_document['regno'] = regno
        user_document["subject-name"] = sub_name
        user_document["exam-date"] = exam_date
        user_document["exam-name"] = testtype
        user_document["class"] = std
        user_document["schoolname"] = schoolname

    else:
        formattedCollection.delete_one({'regno': regno})

    if part == None:
        user_document[section] = formatted_document
    else:
        user_document[section][part] = formatted_document

    formattedCollection.insert_one(user_document)

    print("Logged {regno} response in formatted Collection".format(regno=regno))

In [152]:

@app.route('/examination/response/create/update', methods=['GET', 'POST'])
def create_or_update_answers():
    response_string = {"status": "failed","response":"restricted access"}
    if request.method == 'POST':
        
        db = myclient["examination"]
#         rawResponseCollection = db["rawResponse-paperII"]
#         formattedResponse = db["formattedResponse-paperII"]
        data = request.get_json()
 
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        exam_date = data['exam-date']
        regno=data['regno']
        
            
        rawResponseCollection = db["rawResponse-paperI"]
        formattedResponse = db["formattedResponse-paperI"]

        user_document = rawResponseCollection.find_one({"regno":data['regno']})

        if user_document == None:
            # inserting new document
            insert_doc = rawResponseCollection.insert_one(data)

            #  START
            user_format_document = data

            regno = user_format_document['regno']

            for i in ["question paper id", "regno", "title", "_id", "timestamp","exam-date","subject-name","class","exam-name","schoolname"]:
                if i in user_format_document.keys():
                    del user_format_document[i]


            for section in user_format_document:
#                 thread = threading.Thread(target=format_function, args=(user_format_document[section], section, None, regno, True, False))
#                 thread.start()
#                 thread.join()
                format_function(sub_name,exam_date,testtype,std,schoolname,user_format_document[section], section, None, regno, True, False)

            # END


            if insert_doc.acknowledged == True:
                response_string = {"status": "success","response":"New document created"}
            else:
                response_string = {"status": "failed","response":"response not acknowledged"}

        else:
            rawResponseCollection.delete_one({"regno":request.json['regno']})
            new_document = data

            # removing unwanted keys in incoming document
            for i in ["question paper id", "regno", "title","exam-date","subject-name","class","exam-name","schoolname"]:
                if i in new_document.keys():
                    del new_document[i]

            user_document['timestamp'] = new_document['timestamp']
            del new_document['timestamp']

            regno = user_document['regno']

            flag = False
            for section in new_document:
                if flag:
                    break
                if section not in user_document.keys():
                    user_document[section] = new_document[section]
                    rawResponseCollection.insert_one(user_document)
#                     thread = threading.Thread(target=format_function, args=(new_document[section], section, None, regno, True, False))
#                     thread.start()
#                     thread.join()
                    format_function(sub_name,exam_date,testtype,std,schoolname,new_document[section], section, None, regno, True, False)
                    flag = True
                    break

                for part in new_document[section]:

                    if flag:
                        break
                    if part not in user_document[section].keys():
                        user_document[section][part] = new_document[section][part]
                        rawResponseCollection.insert_one(user_document)
#                         thread = threading.Thread(target=format_function, args=(new_document[section][part], section, part, regno, False, True))
#                         thread.start()
#                         thread.join()
                        format_function(sub_name,exam_date,testtype,std,schoolname,new_document[section][part], section, part, regno, False, True)
                        flag = True
                        break

            response_string = {"status": "success","response":"Document updated"}

        return json.dumps(response_string)


In [153]:
'''
POST:
    description:
        It searches for the word in the user document.
    database:
        name : examination
        collection : rawResponse
    parameters:
        regno : string
    return:
        200:
            Path of the word searched.
        404:
            (1). Document not found.
            (2). Word doesn't exist in the document
'''

@app.route('/examination/answer/search', methods = ['GET', 'POST'])
def answer_search():
    
    answersCollection = myclient["examination"]["rawResponse-paperI"]
    
    if request.method == 'POST':
        data = request.get_json()
        search_word = data['search word']
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno = data['regno']
        

        
        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":

            answer_document=answersCollection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

        else:
            answer_document=answersCollection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype,"schoolname":schoolname},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

#             answer_document = answersCollection.find_one({'regno': regno}, {'_id':0, 'timestamp':0, 'regno':0,"question paper id":0,"title":0})

        if answer_document is None:
            return make_response(json.dumps({"status": "failed", "message": "No answer found for user with regno {r}".format(r=regno)}), 404)
        else:
            matches = []
            final_ans = []
            for section in answer_document:
                for part in answer_document[section]:
                    for question in answer_document[section][part]:

                        final_ans = answer_document[section][part][question]["answer"]  # the answer array for the user
                        
                        if final_ans !=None:
                                        # Converting to lower case for better matching
                            
                            if len(final_ans) == 1:
                                try:
#                                     if final_ans[0] != None:
                                    idx = final_ans[0].index(search_word.lower())
                                    matches.append({"section": section, "part": part, "question": question, "answer index": idx})
                                except:
                                    pass
                            else:
                                for index in range(len(final_ans)):
                                    if final_ans[index] != None:
                                        if search_word.lower() in final_ans[index]:
                                            matches.append({"section": section, "part": part, "question": question, "answer index": index})

            if len(matches) == 0:
                return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document with regno {r}".format(r=regno)}), 200)
            else:
                return make_response(json.dumps({"status": "success", "matches": matches}), 200)

In [154]:
'''
GET:
    parameter: 
        regno->str
    return value:
        user_document->dict
        
'''
@app.route('/examination/response/review-page', methods=['GET', 'POST'])
def review_page():
    if request.method == 'GET':
        data = request.get_json()
        print(data)
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno = data['regno']
        
            
        collection = myclient["examination"]["formattedResponse-paperI"]

        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":

            user_document = collection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

        else:
            user_document = collection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype,"schoolname":schoolname},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

        if user_document != None:

            return make_response(json.dumps({'regno': regno, 'answer':user_document}), 200)

        else:

            return make_response(json.dumps({'regno': regno, 'message':'User document not found or processing in progress'}), 404)

    return make_response(json.dumps({'message':'Forbidden Access'}), 403)

In [155]:
'''
    POST:
        parameter:
            user_document->dict
        return:
            status message from server (Response 200)
'''

@app.route('/examination/response/final/submission',methods=["POST", "GET"])
def final_paper_submission():
    if request.method == "POST":
        
        user_document = request.get_json()
        exam_date = user_document['date of exam']
        sub_name = user_document['title']
        std = user_document["standard"]
        testtype = user_document['type of test']
        schoolname = user_document['school name']
        regno = user_document['regno']
        user_document['tag'] = "final"

        answerCollection = myclient["examination"]["Answersheet"]
        if answerCollection != None:
            answerCollection.insert_one(user_document)
            return make_response(json.dumps({"status": "success", "message":"Submission received"}),200)
    else:
        return make_response(json.dumps({"status": "failed", "message":"Forbidden Access"}),403)

In [156]:
'''
POST:
    description:
        It searches for the word in the user document.
    database:
        name : examination
        collection : rawResponse
    parameters:
        regno : string
    return:
        200:
            Path of the word searched.
        404:
            (1). Document not found.
            (2). Word doesn't exist in the document
'''


@app.route('/examination/questions/search', methods = ['GET', 'POST'])
def questions_search():
    
    quesCollection = myclient["English-II-QuestionBank"]["English-II-Paper"]
    
    if request.method == 'POST':
        
         
        data = request.get_json()
        search_word = data['search word']
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno=data['regno']
        
        if "english" in sub_name.lower() and "II" in sub_name:
            
            if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
                doc = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype},{'_id':0,"school name":0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})
                
            else:
                doc = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname},{'_id':0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})
        
            if doc is None:
                
                return make_response(json.dumps({"status": "failed", "message": "No question found"}), 404)
            
            else:
                occurences = []

                del doc["date of exam"]
                del doc["standard"]
                del doc["type of test"]
                if "school name" in doc:
                    del doc["school name"]
                
                for section in doc:
                    parts = doc[section]
                    if "title" in parts:
                        del parts["title"]
                    if "marks" in parts:
                        del parts["marks"]
                    if "heading" in parts:
                        del parts["heading"]
                    for part in parts:
                        if "marks" in parts[part]:
                            del parts[part]["marks"]
                        if "heading" in parts[part]:
                            del parts[part]["heading"]

                        for ques in parts[part]:
                            ques_data = parts[part][ques]

                            if ques_data != None:
                                if "paragraph" in ques_data.keys() and ques_data["paragraph"]!=None:
                                    if ques_data["paragraph"][0].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                        
                                    elif ques_data["paragraph"][0].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})

                                if "heading" in ques_data.keys() and ques_data["heading"]!=None:
                                    
                                    if ques_data["heading"].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                        
                                    elif ques_data["heading"].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})

                                if "question" in ques_data.keys():
                                    if type(ques_data["question"]) == dict:
                                        for key in ques_data["question"]:
                                            if type(ques_data["question"][key]) == dict:
                                                 if "question" in ques_data["question"][key].keys():
                                                    if type(ques_data["question"][key]["question"]) == dict:  
                                                        if ques_data["question"][key]["question"].lower().find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques})
                                                        elif ques_data["question"][key]["question"].find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques})

                                                    if type(ques_data["question"][key]["question"]) == str:  
                                                        if ques_data["question"][key]["question"].lower().find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})
                                                        elif ques_data["question"][key]["question"].find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})

                                            if type(ques_data["question"][key]) == str:
                                                if ques_data["question"][key].lower().find(search_word) != -1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})
                                                    
                                                elif ques_data["question"][key].find(search_word) != -1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

                                                
                                    if type(ques_data["question"]) == str:

                                            if ques_data["question"].lower().find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques})
                                            elif ques_data["question"].find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques})

                                    if type(ques_data["question"]) == list:
                                            for index in range(len(ques_data["question"])):
                                                if ques_data["question"][index].lower().find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":(index+1)})
                                                    
                                                elif ques_data["question"][index].find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":(index+1)})


                                if "questions" in ques_data.keys():
                                    if type(ques_data["questions"]) == dict:
                                        for key in ques_data["questions"]:
                                            if type(ques_data["questions"][key]) == dict: 
                                                if "question" in ques_data["questions"][key].keys():
                                                    if type(ques_data["questions"][key]["question"]) == str:
                                                        if ques_data["questions"][key]["question"].lower().find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})
                                                        elif ques_data["questions"][key]["question"].find(search_word)!=-1:
                                                            occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

                                                if "heading" in ques_data["questions"][key].keys() and ques_data["questions"][key]["heading"] != None:
                                                    if ques_data["questions"][key]["heading"].lower().find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})
                                                    elif ques_data["questions"][key]["heading"].find(search_word)!=-1:
                                                        occurences.append({"section": section, "part": part, "question": ques,"subquestion": key})

                                            if type(ques_data["questions"][key]) == str:
                                                if ques_data["questions"][key].lower().find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})
                                                if ques_data["questions"][key].find(search_word)!=-1:
                                                    occurences.append({"section": section, "part": part, "question": ques,"subquestion":key})

                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)
                
        else:
            
            quesCollection = myclient["English-I-QuestionBank"]["EnglishQuestionPaper"]
            
            if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":
                doc1 = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype},{'_id':0,"school name":0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})
                
            else:
                doc1 = quesCollection.find_one({"date of exam":exam_date,"standard":std,"type of test":testtype,"school name":schoolname},{'_id':0,"title":0,"time":0,"maximum marks":0,"user":0,"submitted":0,"instructions":0})

            if doc1 is None:
                return make_response(json.dumps({"status": "failed", "message": "No question found"}), 404)
            
            else:
                occurences = []
    #             search_word = "Read the passage given below and answer the questions that follow"
                del doc1["date of exam"]
                del doc1["standard"]
                del doc1["type of test"]
                if "school name" in doc1:
                    del doc1["school name"]

                for section in doc1:
                    parts = doc1[section]
                    if "title" in parts:
                        del parts["title"]
                    if "marks" in parts:
                        del parts["marks"]
                    if "heading" in parts:
                        del parts["heading"]
                    for part in parts:

                        if "marks" in parts[part]:
                            del parts[part]["marks"]

                        if "heading" in parts[part] and parts[part]["heading"] != None:
                            if parts[part]["heading"].lower().find(search_word)!=-1:
                                occurences.append({"section": section, "part": part})
                            elif parts[part]["heading"].find(search_word)!=-1:
                                occurences.append({"section": section, "part": part})
                                
                            del parts[part]["heading"]

                        for ques in parts[part]:

                            ques_data = parts[part][ques]

                            if ques_data != None:

                                if "paragraph" in ques_data.keys() and ques_data["paragraph"]!=None:
                                    if ques_data["paragraph"][0].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                    elif ques_data["paragraph"][0].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})

                                if "heading" in ques_data.keys() and ques_data["heading"]!=None:
                                    if ques_data["heading"].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                    elif ques_data["heading"].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})

                                if "quotedWords" in ques_data.keys() and ques_data["quotedWords"] != None:
                                    if ques_data["quotedWords"].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                    elif ques_data["quotedWords"].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})

                                if "question" in ques_data.keys() and ques_data["question"]!=None:

                                    if type(ques_data["question"]) == dict:
                                        for key in ques_data["question"]:
                                            if ques_data["question"][key].lower().find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques, "question key": key})
                                            elif ques_data["question"][key].find(search_word)!=-1:
                                                occurences.append({"section": section, "part": part, "question": ques, "question key": key})

                                    elif ques_data["question"].lower().find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                        
                                    elif ques_data["question"].find(search_word)!=-1:
                                        occurences.append({"section": section, "part": part, "question": ques})
                                    

                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)


In [157]:
@app.route('/print-II-pdf',methods=["POST"])
def print_II_pdf():
    db=myclient["English-II-QuestionBank"]
    collection=db["English-II-Paper"]
    if request.method=='POST':
        db_response=collection.find_one({},{"_id":0})
     
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "failed","response": "Question not found"})

In [158]:
@app.route('/content/guide/search', methods = ['GET', 'POST'])
def content_guide_search():
    
    
    
    if request.method == 'POST':
        
        search_word = request.form['search word']
        
        ch_no = request.form['chapter no']
        
        name = request.form['name']
        
        cont_type = request.form['unittype']
        
        if cont_type == 'prose':
            
            proseCollection = myclient["contentsDB"]["prose-guide"]
       
            contentdoc = (proseCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []

                for chapter in contentdoc:

                    content = contentdoc[chapter]

                    for key in content:

                        for ques in content[key]:

                            if ques.lower().find(search_word) != -1:

                                occurences.append({"heading no":key,"heading":ques})
                                

                            ques_data = content[key][ques]

                            if ques_data != None:
                                
                                if "heading" in ques_data.keys() and ques_data["heading"] != None:

                                    if ques_data["heading"].lower().find(search_word)!=-1:
                                        occurences.append({"heading no": key,"heading":ques})
                                    elif ques_data["heading"].find(search_word)!=-1:
                                        occurences.append({"heading no": key,"heading":ques})

                                if "question" in ques_data.keys() and ques_data["question"] != None:
                                    

                                    if type(ques_data["question"]) == dict:
                                            for k in ques_data["question"]:
                                                if ques_data["question"][k].lower().find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question no": k,"question":ques_data["question"][k]})
                                                elif ques_data["question"][k].find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question no": k,"question":ques_data["question"][k]})

                                    elif type(ques_data["question"]) == str:
                                            if ques_data["question"].lower().find(search_word)!=-1:
                                                occurences.append({"heading no": key, "heading": ques,"question":ques_data["question"]})
                                            elif ques_data["question"].find(search_word)!=-1:
                                                occurences.append({"heading no": key, "heading": ques,"question":ques_data["question"]})

                                if "answer" in ques_data.keys() and ques_data["answer"] != None: 
                                    

                                    if type(ques_data["answer"]) == dict:
                                            for k in ques_data["answer"]:
                                                if ques_data["answer"][k].lower().find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question no": k})
                                                elif ques_data["answer"][k].find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question no": k})

                                    elif type(ques_data["answer"]) == str:
                                        
                                            if ques_data["question"] != "":
                        
                                                if ques_data["answer"].lower().find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question":ques_data["question"]})
                                                elif ques_data["answer"].find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"question":ques_data["question"]})
                                            else:
                                                
                                                if ques_data["answer"].lower().find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"answer":ques_data["answer"]})
                                                elif ques_data["answer"].find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"answer":ques_data["answer"]})
                                                    
                                con_data = ques_data
                                
                                for i in ["question", "heading", "answer"]:
                                        if i in con_data.keys():
                                            del con_data[i]
                                
                                for num in con_data:
                                    
                                    content_data = con_data[num]
                                    
                                        
                                    if "heading" in content_data.keys() and content_data["heading"] != None:

                                        if content_data["heading"].lower().find(search_word)!=-1:
                                            occurences.append({"heading no": key,"heading":ques,"sub heading no":num,"sub heading":content_data["heading"]})
                                        elif content_data["heading"].find(search_word)!=-1:
                                            occurences.append({"heading no": key,"heading":ques,"sub heading no":num,"sub heading":content_data["heading"]})

                                    if "question" in content_data.keys() and content_data["question"] != None:


                                        if type(content_data["question"]) == dict:
                                                for k in content_data["question"]:
                                                    if content_data["question"][k].lower().find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question no": k,"question":content_data["question"][k]})
                                                    elif content_data["question"][k].find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question no": k,"question":content_data["question"][k]})

                                        elif type(content_data["question"]) == str:
                                                if content_data["question"].lower().find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question":content_data["question"]})
                                                elif content_data["question"].find(search_word)!=-1:
                                                    occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question":content_data["question"]})

                                    if "answer" in content_data.keys() and content_data["answer"] != None: 


                                        if type(content_data["answer"]) == dict:
                                                for k in content_data["answer"]:
                                                    if content_data["answer"][k].lower().find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question no": k})
                                                    elif content_data["answer"][k].find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question no": k})

                                        elif type(content_data["answer"]) == str:

                                                if content_data["question"] != "":

                                                    if content_data["answer"].lower().find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question":content_data["question"]})
                                                    elif content_data["answer"].find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"question":content_data["question"]})
                                                else:

                                                    if content_data["answer"].lower().find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"answer":content_data["answer"]})
                                                    elif content_data["answer"].find(search_word)!=-1:
                                                        occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"answer":content_data["answer"]})

                                    if "passage" in content_data.keys() and content_data["passage"] != None and content_data["passage"] !="":

                                        if content_data["passage"].lower().find(search_word)!=-1:
                                                occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"passage":content_data["passage"]})
                                        elif content_data["passage"].find(search_word)!=-1:
                                                occurences.append({"heading no": key, "heading": ques,"sub heading no":num,"passage":content_data["passage"]})

                    

                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

        if cont_type == 'poem':
            
            poemCollection = myclient["contentsDB"]["poem-guide"]
            
            contentdoc = (poemCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []
                
                
                    
                if "questions" in contentdoc.keys() and contentdoc["questions"] != None:

                    if type(contentdoc["questions"]) == dict:

                        for key in contentdoc["questions"]:

                            content = contentdoc["questions"][key]

                            if "heading" in content.keys() and content["heading"] != None:

                                if content["heading"].lower().find(search_word) != -1:

                                    occurences.append({"heading":"questions","sub heading no":key,"sub heading":content["heading"]})

                                elif content["heading"].find(search_word) != -1:

                                    occurences.append({"heading":"questions","sub heading no":key,"sub heading":content["heading"]})

                            if "question" in content.keys() and content["question"] != None:

                                if type(content["question"]) == dict:

                                    for num in content["question"]:

                                        if content["question"][num].lower().find(search_word) != -1:

                                            occurences.append({"heading":"questions","sub heading no":key,"question no":num})

                                        elif content["question"][num].find(search_word) != -1:

                                            occurences.append({"heading":"questions","sub heading no":key,"question no":num})
                                            
                            if "answer" in content.keys() and content["answer"] != None:

                                if type(content["answer"]) == dict:

                                    for num in content["answer"]:

                                        if content["answer"][num].lower().find(search_word) != -1:

                                            occurences.append({"heading":"questions","sub heading no":key,"question no":num})

                                        elif content["answer"][num].find(search_word) != -1:

                                            occurences.append({"heading":"questions","sub heading no":key,"question no":num})
                
                if "summary" in contentdoc.keys() and contentdoc["summary"] != None:
                   
                    content = contentdoc["summary"]
                    
                    if "heading" in content.keys() and content["heading"] != None:

                        if content["heading"].lower().find(search_word) != -1:

                            occurences.append({"heading":"summary","sub heading":content["heading"]})

                        elif content["heading"].find(search_word) != -1:

                            occurences.append({"heading":"summary","sub heading":content["heading"]})

                    if "question" in content.keys() and content["question"] != None:

                        if content["question"].lower().find(search_word) != -1:

                            occurences.append({"heading":"summary","question":content["question"]})

                        elif content["question"].find(search_word) != -1:

                            occurences.append({"heading":"summary","question":content["question"]})

                    if "answer" in content.keys() and content["answer"] != None:


                        if content["answer"].lower().find(search_word) != -1:

                            occurences.append({"heading":"questions","question":content["question"]})

                        elif content["answer"].find(search_word) != -1:

                            occurences.append({"heading":"questions","question":content["question"]})
                            
                            
                if "glossary" in contentdoc.keys() and contentdoc["glossary"] != None:
                   
                    content = contentdoc["glossary"]
                    
                    if "heading" in content.keys() and content["heading"] != None:

                        if content["heading"].lower().find(search_word) != -1:

                            occurences.append({"heading":"glossary","sub heading":content["heading"]})

                        elif content["heading"].find(search_word) != -1:

                            occurences.append({"heading":"glossary","sub heading":content["heading"]})

                    if "question" in content.keys() and content["question"] != None:

                        if content["question"].lower().find(search_word) != -1:

                            occurences.append({"heading":"glossary","question":content["question"]})

                        elif content["question"].find(search_word) != -1:

                            occurences.append({"heading":"glossary","question":content["question"]})

                    if "answer" in content.keys() and content["answer"] != None:

                        if content["answer"].lower().find(search_word) != -1:

                            occurences.append({"heading":"glossary","question":content["question"]})

                        elif content["answer"].find(search_word) != -1:

                            occurences.append({"heading":"glossary","question":content["question"]})


                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

                
        if cont_type == 'supplementary':
            
            supplyCollection = myclient["contentsDB"]["supplementary-guide"]
            
            contentdoc = (supplyCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []


                if "Questions" in contentdoc.keys() and contentdoc["Questions"] != None:
        
                    for index in contentdoc["Questions"]:
                
                        if contentdoc["Questions"][index] != None:

                            content = contentdoc["Questions"][index]
                            
                            if "Headings" in content.keys() and content["Headings"] !=None:

                                if content["Headings"].lower().find(search_word) != -1:
                                    
                                    occurences.append({"heading no": index, "heading": content["Headings"]}) 

                                elif content["Headings"].find(search_word) != -1:

                                    occurences.append({"heading no": index, "heading": content["Headings"]}) 

                            if "question" in content.keys() and content["question"] !=None:

                                for ques_no in content["question"]:

                                    if content["question"][ques_no] != None:

                                        if content["question"][ques_no].lower().find(search_word) != -1:
                                            

                                            occurences.append({"heading no": index, "heading": content["Headings"],"question no":ques_no,"question":content["question"][ques_no]}) 

                                        elif contentdoc["Questions"][index]["question"][ques_no].find(search_word) != -1:

                                            occurences.append({"heading no": index, "heading": content["Headings"],"question no":ques_no,"question":content["question"][ques_no]}) 

                            if "answer" in content.keys() and content["answer"] !=None:
                                
                                if type(content["answer"]) == dict:

                                    for ans_no in content["answer"]:

                                     
                                        if content["answer"][ans_no].lower().find(search_word) != -1:
                                        

                                            occurences.append({"heading no": index, "heading": content["Headings"],"question no":ans_no,"question":content["question"][ans_no],"answer":content["answer"][ans_no]}) 

                                        elif content["answer"][ans_no].find(search_word) != -1:

                                            occurences.append({"heading no": index, "heading": content["Headings"],"question no":ans_no,"question":content["question"][ans_no],"answer":content["answer"][ans_no]}) 
                            
                        

                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)
                
        if cont_type == 'grammar':
            
            grammarCollection = myclient["contentsDB"]["grammar"]
            
            contentdoc = (grammarCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []

                if "Questions" in contentdoc.keys() and contentdoc["Questions"] != None:
        
                    for index in contentdoc["Questions"]:
                
                        content = contentdoc["Questions"][index]

                        if "heading" in content.keys() and content["heading"] !=None:

                            if content["heading"].lower().find(search_word) != -1:

                                occurences.append({"heading no": index, "heading": content["heading"]}) 

                            elif content["heading"].find(search_word) != -1:

                                occurences.append({"heading no": index, "heading": content["heading"]}) 

                        if "question" in content.keys() and content["question"] !=None:

                            if type(content["question"]) == dict:

                                for ques_no in content["question"]:

                                    if content["question"][ques_no] != None:

                                        if content["question"][ques_no].lower().find(search_word) != -1:


                                            occurences.append({"heading no": index, "heading": content["heading"],"question no":ques_no,"question":content["question"][ques_no]}) 

                                        elif content["question"][ques_no].find(search_word) != -1:

                                            occurences.append({"heading no": index, "heading": content["heading"],"question no":ques_no,"question":content["question"][ques_no]}) 

                            elif type(content["question"]) == str:

                                if content["question"].lower().find(search_word) != -1:

                                    occurences.append({"heading no": index, "heading": content["heading"],"question":content["question"]}) 

                                elif content["question"].find(search_word) != -1:

                                    occurences.append({"heading no": index, "heading": content["heading"],"question":content["question"]}) 


                        if "answer" in content.keys() and content["answer"] !=None:
                            
                            if type(content["answer"]) == dict:

                                for ans_no in content["answer"]: 
                                    
                                    if content["answer"][ans_no].lower().find(search_word) != -1:


                                        occurences.append({"heading no": index, "heading": content["heading"],"question no":ans_no,"question":content["question"][ans_no],"answer":content["answer"][ans_no]}) 

                                    elif content["answer"][ans_no].find(search_word) != -1:

                                        occurences.append({"heading no": index, "heading": content["heading"],"question no":ans_no,"question":content["question"][ans_no],"answer":content["answer"][ans_no]}) 


                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

        

In [159]:
@app.route('/content/lesson/search', methods = ['GET', 'POST'])
def content_lesson_search():
    

    if request.method == 'POST':
        
        search_word = request.form['search word']
        
        ch_no = request.form['chapter no']
        
        name = request.form['name']
        
        cont_type = request.form['unittype']
        
        if cont_type == 'prose':
            
            proseCollection = myclient["contentsDB"]["proses"]
       
            contentdoc = (proseCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []
                
                sentences = []
                
                if "contents" in contentdoc.keys() and contentdoc["contents"] !=None:

                    content = contentdoc["contents"]

                    content = content.split('.')
                    

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                if "author" in contentdoc.keys() and contentdoc["author"] != None:
                    
                    if "about the author" in contentdoc["author"].keys() and contentdoc["author"]["about the author"] != None:
                          
                        content = contentdoc["author"]["about the author"]
                        
                        content = content.split('.')
                        
                        for index in range(len(content)):
                        
                            sentences.append(content[index])
                            
                            if search_word in content[index].lower():
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})

                            elif search_word.lower() in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                            elif search_word in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                if "glossary" in contentdoc.keys() and contentdoc["glossary"] !=None:

                    content = contentdoc["glossary"]

                    content = content.split('\n')
                    

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                        
#                 print(sentences)

#                 for index in range(len(sentences)):
#                     if search_word in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})

#                     elif search_word.lower() in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})
                
                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

        if cont_type == 'poem':
            
            poemCollection = myclient["contentsDB"]["poems"]
            
            contentdoc = (poemCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []
                
                
                sentences = []
                
                if "contents" in contentdoc.keys() and contentdoc["contents"] !=None:

                    content = contentdoc["contents"]

                    content = content.split('\n')
                    

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                if "author" in contentdoc.keys() and contentdoc["author"] != None:
                    
                    if "about the author" in contentdoc["author"].keys() and contentdoc["author"]["about the author"] != "":
                          
                        content = contentdoc["author"]["about the author"]
                        
                        content = content.split('.')
                        
                        for index in range(len(content)):
                        
                            sentences.append(content[index])
                            
                            if search_word in content[index].lower():
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})

                            elif search_word.lower() in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                            elif search_word in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                if "glossary" in contentdoc.keys() and contentdoc["glossary"] !="":

                    content = contentdoc["glossary"]

                    content = content.split('\n')

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                if "description" in contentdoc.keys() and contentdoc["description"] !="":

                    content = contentdoc["description"]

                    content = content.split('.')

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Description","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Description","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Dlossary","Index":index,"sentence":content[index]})
                        
#                 print(sentences)

#                 for index in range(len(sentences)):
#                     if search_word in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})

#                     elif search_word.lower() in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})


                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

                
        if cont_type == 'supplementary':
            
            supplyCollection = myclient["contentsDB"]["supplementaries"]
            
            contentdoc = (supplyCollection.find_one({"name":name,"chapter no":int(ch_no)},{"_id":0,"name":0,"chapter no":0}))

            if contentdoc is None:

                return make_response(json.dumps({"status": "failed", "message": "Lesson not found"}), 404)

            else:
                occurences = []
                
                sentences = []
                
                if "contents" in contentdoc.keys() and contentdoc["contents"] !=None:

                    content = contentdoc["contents"]

                    content = content.split('.')
                    

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Lesson","Index":index,"sentence":content[index]})
                            
                if "author" in contentdoc.keys() and contentdoc["author"] != None:
                    
                    if "about the author" in contentdoc["author"].keys() and contentdoc["author"]["about the author"] != "":
                          
                        content = contentdoc["author"]["about the author"]
                        
                        content = content.split('.')
                        
                        for index in range(len(content)):
                        
                            sentences.append(content[index])
                            
                            if search_word in content[index].lower():
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})

                            elif search_word.lower() in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                            elif search_word in content[index]:
                                occurences.append({"heading":"About the author","Index":index,"sentence":content[index]})
                                
                if "glossary" in contentdoc.keys() and contentdoc["glossary"] !="":

                    content = contentdoc["glossary"]

                    content = content.split('\n')

                    for index in range(len(content)):
                         
                            sentences.append(content[index])
                            if search_word.lower() in content[index]:
                                occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})

                            elif search_word in content[index]:
                                occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})

                            elif search_word in content[index].lower():
                                occurences.append({"heading":"Glossary","Index":index,"sentence":content[index]})
                            
                if "description" in contentdoc.keys() and contentdoc["description"] !="":

                    content = contentdoc["description"]

                    content = content.split('.')

                    for index in range(len(content)):
                        
                        sentences.append(content[index])
                        if search_word.lower() in content[index]:
                            occurences.append({"heading":"Description","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index]:
                            occurences.append({"heading":"Description","Index":index,"sentence":content[index]})
                            
                        elif search_word in content[index].lower():
                            occurences.append({"heading":"Dlossary","Index":index,"sentence":content[index]})
                        
#                 print(sentences)

#                 for index in range(len(sentences)):
#                     if search_word in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})

#                     elif search_word.lower() in sentences[index]:
#                         occurences.append({"Index":index,"sentence":sentences[index]})
                
                if len(occurences) == 0:
                    return make_response(json.dumps({"status": "failed", "message": "No such word exists in the user document"}), 200)
                else:
                    return make_response(json.dumps({"status": "success", "matches": occurences}), 200)

        

# DISCUSSION FORUM

In [160]:
@app.route('/content/get/response', methods = ['POST'])
def content_get_responseDB():
    if request.method == 'POST':
        collection = myclient["Class"]["discussion forum"]
        requestData = request.get_json()
        regno = requestData["regno"]
        std = requestData["class"]
        sec = requestData["section"]
        s_name = requestData["schoolname"]
        
        db_response = collection.find_one({"regno": regno,"class":std,"section":sec,"schoolname":s_name},{"_id":0})
        
        if db_response != None:
            response_string = {"status":"success","response":db_response}
            
        else:
            response_string = {"status":"success","response":"not found"}
        
        return make_response(json.dumps(response_string),200)

In [161]:
@app.route('/content/send/request', methods = ['GET', 'POST'])
def content_send_requestUI():
    if request.method == 'POST':
        collection = myclient["Class"]["discussion forum"]
        requestData = request.get_json()
        regno = requestData["regno"]
        std = requestData["class"]
        sec = requestData["section"]
        s_name = requestData["schoolname"]
        
        db_response = collection.find_one({"regno": regno,"class":std,"section":sec,"schoolname":s_name})
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"clarifications":requestData["clarifications"]}})
            if update_result.acknowledged == True:
                response_string = {"status":"success","response":"request updated"}
            
        else:
            insert_response = collection.insert_one(requestData)
            if insert_response.acknowledged == True:
                response_string = {"status":"success","response":"request sent"}
        
        return make_response(json.dumps(response_string),200)

# TEACHER' NOTES - RETRIEVAL

In [162]:
@app.route('/get/teacher/notes',methods=["POST"])
def get_teacher_notes():
    db=myclient["English-TeachernotesQuestionBank"]
    
    data = request.get_json()
    
    regno = data["regno"]
    
    sub = data["subject"]
    
    std = data["class"]
    
    std = std+"th Standard"
    
    sec = data["section"]
    
    sch_name = data["schoolname"]
    
    ch_no = data["chapter-no"]
    
    unit_type = data["chapter-type"]
    
    match = {}
    
    if request.method=='POST':
        
        if unit_type == "prose":
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_response_student=collection_student.find_one({"regno":regno,"schoolname":sch_name,"class":data["class"],"section":sec},{"_id":0,"Exam Attending":0})
           
            if db_response_student != None:
               
                collection=db["prose - notes"]
                dbs = myclient["accounts"]
                col_response = dbs["teacher"]
                db_res = col_response.find({"schoolname":sch_name},{"_id":0})
                
                for key in db_res:
                    
                    enroll_course = key["enrolledCourses"]
                    
                    regId = key["regId"]
                    
                    for val in enroll_course:
                        if data["class"] == val:
                            for k in enroll_course[val]:
                                if sec == k:                               
                                    db_response = collection.find_one({"user":regId,"chapter-no":int(ch_no),"chapter-type":unit_type,"subject":sub,"standard":std,"section":sec})
                                    if db_response != None:
                                        match = db_response

        if unit_type == "poem":
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_response_student=collection_student.find_one({"regno":regno,"schoolname":sch_name,"class":data["class"],"section":sec},{"_id":0,"Exam Attending":0})
           
            if db_response_student != None:
               
                collection=db["poem - notes"]
                dbs = myclient["accounts"]
                col_response = dbs["teacher"]
                db_res = col_response.find({"schoolname":sch_name},{"_id":0})
                
                for key in db_res:
                    
                    enroll_course = key["enrolledCourses"]
                    
                    regId = key["regId"]
                    
                    for val in enroll_course:
                        if data["class"] == val:
                            for k in enroll_course[val]:
                                if sec == k:                               
                                    db_response = collection.find_one({"user":regId,"chapter-no":int(ch_no),"chapter-type":unit_type,"subject":sub,"standard":std,"section":sec})
                                    if db_response != None:
                                        match = db_response
                                
                                
        if unit_type == "supplementary":
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_response_student=collection_student.find_one({"regno":regno,"schoolname":sch_name,"class":data["class"],"section":sec},{"_id":0,"Exam Attending":0})
           
            if db_response_student != None:
               
                collection=db["supplementary - notes"]
                dbs = myclient["accounts"]
                col_response = dbs["teacher"]
                db_res = col_response.find({"schoolname":sch_name},{"_id":0})
                
                for key in db_res:
                    
                    enroll_course = key["enrolledCourses"]
                    
                    regId = key["regId"]
                    
                    for val in enroll_course:
                        if data["class"] == val:
                            for k in enroll_course[val]:
                                if sec == k:                               
                                    db_response = collection.find_one({"user":regId,"chapter-no":int(ch_no),"chapter-type":unit_type,"subject":sub,"standard":std,"section":sec})
                                    if db_response != None:
                                        match = db_response
                                
        if unit_type == "grammar":
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_response_student=collection_student.find_one({"regno":regno,"schoolname":sch_name,"class":data["class"],"section":sec},{"_id":0,"Exam Attending":0})
           
            if db_response_student != None:
               
                collection=db["grammar - notes"]
                dbs = myclient["accounts"]
                col_response = dbs["teacher"]
                db_res = col_response.find({"schoolname":sch_name},{"_id":0})
                
                for key in db_res:
                    
                    enroll_course = key["enrolledCourses"]
                    
                    regId = key["regId"]
                    
                    for val in enroll_course:
                        if data["class"] == val:
                            for k in enroll_course[val]:
                                if sec == k:                               
                                    db_response = collection.find_one({"user":regId,"chapter-no":int(ch_no),"chapter-type":unit_type,"subject":sub,"standard":std,"section":sec})
                                    if db_response != None:
                                        match = db_response

        if len(match) == 0:
            return json.dumps({"status": "failed","response": "Notes not found"})
            
        else:
            return json_util.dumps(match)


# Update content details

In [163]:
@app.route('/update/fileDetails',methods=["POST"])
def update_fileDetails():
#   declaration space
    db = myclient["Class"]
    collection = db["contents"]
    data = {
        "regId" : request.form["regId"],
        "createdAt" : request.form["createdAt"],
#         "filename" : request.form["file_name"],
        "class" : request.form["class"],
        "section" : request.form["section"],
        "schoolname" : request.form["schoolname"],
        "chapter_no" : request.form["chapter_no"],
        "unit_type":request.form["unit_type"],
        "chapter_name": request.form["name"]
        
    }
    
    
    response_string = {}
    if request.method == 'POST':
        
        db_response = collection.find_one({"regId":data["regId"],"class":data["class"],"section":data["section"],"schoolname":data["schoolname"],"chapter_no":data["chapter_no"],"unit_type":data["unit_type"],"chapter_name":data["chapter_name"]})
        
        if db_response == None:
            db_res = collection.insert_one(data)

            if db_res.acknowledged == True:
                response_string = {"status": "success","response":"details inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        else:
            delete_file = collection.delete_one({"_id":ObjectId(db_response["_id"])})
            
            if delete_file.acknowledged == True:
                
                db_res = collection.insert_one(data)

                if db_res.acknowledged == True:
                    response_string = {"status": "success","response":"details updated"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}
                
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
                
    return json.dumps(response_string)

In [164]:
@app.route('/filesData/getAll',methods=["GET", "POST"])
def files_get():
    db = myclient["Class"]
    collection = db["contents"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "user not found","response": "User not found"})

# ASSIGNMENT

In [165]:
# @app.route('/teacher/post-assignments',methods=["POST"])
# def insert_assignments():
# #   declaration space
#     db = myclient["Class"]
#     collection = db["assignments"]
#     response_string = {}
#     if request.method == 'POST':
#         incoming_data = json.loads(request.form['name'])
#         insert_response = collection.insert_one(incoming_data)
#         if insert_response.acknowledged == True:
#             response_string = {"status": "success","response":"ok"}
#         else:
#             response_string = {"status":"failed","response":"database error. request not acknowledged"}
#     return make_response(jsonify(response_string),200)
@app.route('/teacher/post-assignments',methods=["POST"])
def insert_assignments():
#   declaration space
    db = myclient["Class"]
    collection = db["assignments"]
    response_string = {}
    if request.method == 'POST':

     
        data = json.loads(request.form['name'])
        
        existing_data = collection.find_one({"user":data["user"],"subject":data["subject"],"class":data["class"],"section":data["section"],"schoolname":data["schoolname"]})
            
        if existing_data == None:
            count = 1
            
           
            data["assign-no"] = str(count)
            count = count + 1
            insert_response = collection.insert_one(data)
            if insert_response.acknowledged == True:
                
                return json.dumps({"status": "success","response":"content inserted"})
            else:
                return json.dumps({"status":"failed","response":"database error. request not acknowledged"})      

        else:
            
            for i in existing_data:
                data["assign-no"] = str(int(existing_data["assign-no"])+1)
                
            update_result =  collection.insert_one(data)
                
            if update_result.acknowledged == True:
            
                return json.dumps({"status":"success","response":"content updated"})


In [166]:
@app.route('/teacher/sslc-course-enrollment',methods=["POST"])
def teacher_sslc_course_enrollment():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        sub = json.loads(request.form['name'])
        regId = sub['regId']
        std = sub['std']
        sec = sub['sec']
        subjects = sub['subjects']
        schoolname = sub['schoolname']
        db_response = collection.find_one({"regId":regId,"schoolname":schoolname})
        courses = db_response['enrolledCourses']
        print(courses)
        
        if db_response != None:
            courses[std][sec] = subjects
            update_result = collection.update_one({"_id":ObjectId(db_response['_id'])},{"$set":{"enrolledCourses":courses}})
            if update_result.acknowledged:
                return json.dumps({"status": "success","response": "ok"})
            return json.dumps({"status": "failed","response": "database error"})
        else:
            response_string = {"status":"Teacher not found","response":"database error. request not acknowledged"}
    return make_response(jsonify(response_string),200)

In [167]:
%run -i update-marks.py

In [168]:
%run -i find-rank.py

# Attendance

In [169]:
%run -i get-attendance.py

In [170]:
%run -i mark-attendance.py

IndentationError: expected an indented block (mark-attendance.py, line 47)

In [171]:
%run -i update-attendance.py

In [172]:
@app.route('/assignment/uploads', methods=['GET', 'POST'])
def assign_upload_file():
    if request.method == 'POST':
        f = request.files['file']
        filename = f.filename
        
        if filename != '':
            file_ext = os.path.splitext(filename)[1]
            if file_ext not in app.config['UPLOAD_EXTENSIONS']:
                return json.dumps({"status": "failed","response": "File Type not accepted"})
        else:
            return json.dumps({"status": "failed","response": "No file contents"})
        
        f.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        
        return json.dumps({"status": "success","response": "uploaded successfully", "data": parse_text(UPLOAD_FOLDER+'/'+filename)})
    
    return json.dumps({"status": "failed","response": "Permission Denied"})

In [173]:
@app.route('/teacher/class-time-table',methods=["POST"])
def teacher_class_time_table():
    db = myclient["Class"]
    collection = db["teacher - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"regId": incoming_data['regId'],"schoolname": incoming_data['schoolname']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"]):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"updated"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)


In [174]:
@app.route('/teacher-time-table/getAll',methods=["GET", "POST"])
def teacher_time_table_getAll():
    db = myclient["Class"]
    collection = db["teacher - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

In [175]:
@app.route('/student/class-time-table',methods=["POST"])
def student_class_time_table():
    db = myclient["Class"]
    collection = db["student - time table"]
    response_string = {}
    if request.method == 'POST':
        incoming_data = json.loads(request.form['name'])
        existing_data = collection.find_one({"user": incoming_data['user'],"schoolname": incoming_data['schoolname'],"class": incoming_data['class'],"section": incoming_data['section']},{"_id": 0})
       
        if existing_data != None:
            if(incoming_data["schoolname"] == existing_data["schoolname"] and incoming_data["class"] == existing_data['class'] and incoming_data["section"] == existing_data['section']):
                collection.delete_many(existing_data)
                collection.insert_one(incoming_data)
                response_string = {"status": "successful","response":"updated"}
            else:
                insert_response = collection.insert_one(incoming_data)
                if insert_response.acknowledged == True:
                    response_string = {"status": "success","response":"ok"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}

        else:
            insert_response = collection.insert_one(incoming_data)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
           
    return make_response(jsonify(response_string),200)


In [176]:
@app.route('/student-time-table/getAll',methods=["GET", "POST"])
def student_time_table_getAll():
    db = myclient["Class"]
    collection = db["student - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

In [177]:
@app.route('/student/class-time-table/getAll',methods=["GET", "POST"])
def student_class_time_table_getAll():
    db = myclient["Class"]
    collection = db["student - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

In [178]:
@app.route('/student/notifications',methods=["POST"])
def student_notifications():
    db = myclient["Class"]
    collection = db["student - notifications"]
    if request.method == 'POST':
        posted_time = request.form['posted-time']
        posted_date = request.form['posted-date']
        std = request.form['std']
        section = request.form['sec']
        subject = request.form['subject']
        schoolname = request.form['schoolname']
        regId = request.form['regId']
        notification = request.form['notification']
        insert_result = collection.insert_one({"regId": regId,"schoolname": schoolname,"subject": subject,"notification":notification,"class":std,"section":std,"posted-date": posted_date,"posted-time":posted_time},{"_id": 0})
        if insert_result.acknowledged == True:
            return json.dumps({"status": "successfully notified","response": "ok"})
        return json.dumps({"status": "failed","response": "database error"})

In [179]:
@app.route('/notifications',methods=["POST"])
def notifications():
    db = myclient["Class"]
    collection = db["notifications"]
    if request.method == 'POST':
        posted_time = request.form['posted-time']
        posted_date = request.form['posted-date']
        schoolname = request.form['schoolname']
        exam_name = request.form['exam-name']
        admin_id = request.form['admin-id']
        notification = request.form['notification']
        insert_result = collection.insert_one({"admin-id": admin_id,"schoolname": schoolname,"exam-name": exam_name,"notification":notification,"posted-date": posted_date,"posted-time":posted_time},{"_id": 0})
        if insert_result.acknowledged == True:
            return json.dumps({"status": "successfully notified","response": "ok"})
        return json.dumps({"status": "failed","response": "database error"})

In [180]:
@app.route('/notifications/getAll',methods=["GET", "POST"])
def notifications_getAll():
    db = myclient["Class"]
    collection = db["notifications"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "No notifications","response": "No notifications found"})

In [181]:
@app.route('/teacher/getCourses',methods=["GET", "POST"])
def teacher_getcourses():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        db_response = (collection.find_one({"regId":regId},{"_id":0,"encrypted_pwd":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [182]:
@app.route('/clarifications/getAll',methods=["GET", "POST"])
def clarifications_get():
    db = myclient["Class"]
    collection = db["discussion forum"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [183]:
@app.route('/discussion-forum/getStudent',methods=["GET", "POST"])
def discussion_forum_get_student():
    db = myclient["Class"]
    collection = db["discussion forum"]
    regno = request.form['regno']
    std = request.form['class']
    section = request.form['section']
    schoolname = request.form['schoolname']
    if request.method == 'POST':
        db_response = collection.find_one({"regno":regno,"class":std,"section":section,"schoolname":schoolname},{'_id':0})
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

In [184]:
@app.route('/clarifications/reply',methods=["GET", "POST"])
def clarifications_reply():
    db = myclient["Class"]
    collection = db["discussion forum"]
    doubt = request.get_json()
    if request.method == 'POST':
        db_response = collection.find_one({"regno": doubt['regno'],"class":doubt['class'],"section":doubt['section'],"schoolname":doubt['schoolname']})
        if db_response != None:
            update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"regno": doubt['regno']},{"$set":{"clarifications":doubt['clarifications']}})
            if update_result.acknowledged == True and update_result.modified_count == 1:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error"}
            return json.dumps(response_string, indent=4)
    return json.dumps({"status": "user not found","response": "User not found"})

In [185]:
@app.route('/get/content',methods=["POST"])
def get_content():
    db=myclient["contentsDB"]
    
    data = request.get_json()
    
    regno = data["regno"]
    
    sub = data["subject"]
    
    std = data["class"]
    
    std = std+"th Standard"
    
    sec = data["section"]
    
    sch_name = data["schoolname"]
    
    
    match = {}
    
    ch_no = 1
    
    if request.method=='POST':
        
      
        db_student=myclient["accounts"]
        collection_student=db_student["student"]
        db_response_student=collection_student.find_one({"regno":regno,"schoolname":sch_name,"class":data["class"],"section":sec},{"_id":0,"Exam Attending":0})

        if db_response_student != None:

            collection1 = db["proses"]
            collection2 = db["poems"]
            collection3 = db["supplementaries"]
            
            col_prose_guide = db["prose-guide"]
            col_poem_guide = db["poem-guide"]
            col_supply_guide = db["supplementary-guide"]
            col_gram_guide = db["grammar"]
            
            prose = {}
            
            poem = {}

            supplementary = {}  

            grammar = {}
            
            prose_guide = {}
            
            poem_guide = {}
            
            supply_guide = {}
            
            content = {}
            
            dbs = myclient["accounts"]
            
            col_response = dbs["teacher"]
            
            db_res = col_response.find({"schoolname":sch_name},{"_id":0})
            

            for key in db_res:

                enroll_course = key["enrolledCourses"]

                regId = key["regId"]

                for val in enroll_course:
                    if data["class"] == val:
                        for k in enroll_course[val]:
                            if sec == k: 
                                
                                
                                db_res1 = collection1.find({},{"_id":0})
                                
                                db_res2 = collection2.find({},{"_id":0})
                                
                                db_res3 = collection3.find({},{"_id":0})
                                
                                db_guide1 = col_prose_guide.find({},{"_id":0})
                                
                                db_guide2 = col_poem_guide.find({},{"_id":0})
                                
                                db_guide3 = col_supply_guide.find({},{"_id":0})
                                
                                db_guide4 = col_gram_guide.find({},{"_id":0})
                                
                                
                                for cont in db_res1:
                                    
                                    prose[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_res2:
                                    
                                    poem[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_res3:
                                    
                                    supplementary[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_guide1:
                                    
                                    prose_guide[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_guide2:
                                    
                                    poem_guide[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_guide3:
                                    
                                    supply_guide[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                ch_no = 1
                                    
                                for cont in db_guide4:
                                    
                                    grammar[ch_no] = cont
                                    
                                    ch_no  += 1
                                    
                                    
                                les_content = {"prose":prose,"poem":poem,"supplementary":supplementary}
                                
                                lesson = {"lesson":les_content}
                                
                                guide_content =  {"prose":prose_guide,"poem":poem_guide,"supplementary":supply_guide,"grammar":grammar}
                                
                                guide = {"Q/A":guide_content}
                                
                                content = [lesson, guide]
                                
                                
                                

        if len(content) == 0:
            return json.dumps({"status": "failed","response": "Notes not found"})
            
        else:
            return json_util.dumps(content)


In [186]:
@app.route('/content/deleteone',methods=["POST"])
def content_deleteone():
    db = myclient["English-TeachernotesQuestionBank"]
    data = {
        "regId" : request.form["regId"],
        "schoolname" : request.form["schoolname"],
        "class" : request.form["class"],
        "section" : request.form["section"],
        "chapter_no" : request.form["chapter_no"],
        "chapter_name": request.form["name"],
        "unit_type":request.form["unit_type"]
        
    }
    
    
    print(data)
    
    if request.method == 'POST':
        
        if data["unit_type"] == "grammar":
            
            collection = db["grammar - notes"]
            
            db_response = collection.find_one({"user":data["regId"],"standard":data["class"],"section":data["section"],"chapter-no":int(data["chapter_no"]),"chapter-name":data["chapter_name"],"chapter-type":data["unit_type"]},{"_id":0})
        
            if db_response != None:
                
                delete_content = collection.delete_one(db_response)
                
                dbs = myclient["Class"]
                
                col_file = dbs["contents"]
                
                delete_file_details = col_file.delete_one(data)
                    
                if delete_content.acknowledged == True and delete_file_details.acknowledged == True :

#                 if delete_content.acknowledged == True:

                    return json.dumps({"status": "success", "response": "Content deleted for the regId "+data["regId"]}) 

            else:

                return json.dumps({"status": "failed","response": "User not found"})
        
        if data["unit_type"] == "prose":
            
            collection = db["prose - notes"]
            
            db_response = collection.find_one({"user":data["regId"],"standard":data["class"],"section":data["section"],"chapter-no":int(data["chapter_no"]),"chapter-name":data["chapter_name"],"chapter-type":data["unit_type"]},{"_id":0})
        
            if db_response != None:
                
                delete_content = collection.delete_one(db_response)
                
                dbs = myclient["Class"]
                
                col_file = dbs["contents"]
                
                delete_file_details = col_file.delete_one(data)
                    
                if delete_content.acknowledged == True and delete_file_details.acknowledged == True :

#                 if delete_content.acknowledged == True:

                    return json.dumps({"status": "success", "response": "Content deleted for the regId "+data["regId"]}) 

            else:

                return json.dumps({"status": "failed","response": "User not found"})
        
        if data["unit_type"] == "poem":
            
            collection = db["poem - notes"]
            
            db_response = collection.find_one({"user":data["regId"],"standard":data["class"],"section":data["section"],"chapter-no":int(data["chapter_no"]),"chapter-name":data["chapter_name"],"chapter-type":data["unit_type"]},{"_id":0})
        
            if db_response != None:
                
                delete_content = collection.delete_one(db_response)
                
                dbs = myclient["Class"]
                
                col_file = dbs["contents"]
                
                delete_file_details = col_file.delete_one(data)
                    
                if delete_content.acknowledged == True and delete_file_details.acknowledged == True :

#                 if delete_content.acknowledged == True:

                    return json.dumps({"status": "success", "response": "Content deleted for the regId "+data["regId"]}) 

            else:

                return json.dumps({"status": "failed","response": "User not found"})
        
        if data["unit_type"] == "supplementary":
            
            collection = db["supplementary - notes"]
            
            db_response = collection.find_one({"user":data["regId"],"standard":data["class"],"section":data["section"],"chapter-no":int(data["chapter_no"]),"chapter-name":data["chapter_name"],"chapter-type":data["unit_type"]},{"_id":0})
        
            if db_response != None:
                
                delete_content = collection.delete_one(db_response)
                
                dbs = myclient["Class"]
                
                col_file = dbs["contents"]
                
                delete_file_details = col_file.delete_one(data)
                    
                if delete_content.acknowledged == True and delete_file_details.acknowledged == True :

#                 if delete_content.acknowledged == True:

                    return json.dumps({"status": "success", "response": "Content deleted for the regId "+data["regId"]}) 

            else:

                return json.dumps({"status": "failed","response": "User not found"})
        
                             
      

In [187]:

# @app.route('/assignment/get',methods=["POST"])
# def assign_get():
    
#     db=myclient["Class"]
#     collection=db["assignments"]
    
#     if request.method=='POST':
#         data = request.get_json()
        
# #         exam_date = data['end-date']
#         sub_name = data['subject']
#         std = data["class"]
        
#         sec  = data['section']
#         schoolname = data['schoolname']
#         regno=data['regno']
        
# #         db_res=collection.find_one({"end-date":exam_date,"subject":sub_name,"class":std,"schoolname":schoolname,"section":sec},{"_id":0})

#         db_res=collection.find_one({"subject":sub_name,"class":std,"schoolname":schoolname,"section":sec},{"_id":0})    

#         if db_res != None:
#             ids=db_res.get('_id')
#             db_student=myclient["accounts"]
#             collection_student=db_student["student"]
#             db_res_student=collection_student.find_one({"regno":regno,"section":sec,"class":std,"schoolname":schoolname})
#             if db_res_student != None:
#                 return json_util.dumps(db_res)
#             else:
#                 return json.dumps({"status":"failed","response":"student not found"})
#         else:
#             return json.dumps({"status": "failed","response": "Question not found"})
        


@app.route('/assignment/get',methods=["POST"])
def assign_get():
    
    db=myclient["Class"]
    collection=db["assignments"]
    
    if request.method=='POST':
        data = request.get_json()
        
#         exam_date = data['end-date']
#         sub_name = data['subject']
        std = data["class"]
        
        sec  = data['section']
        schoolname = data['schoolname']
        regno=data['regno']
        
#         db_res=collection.find_one({"end-date":exam_date,"subject":sub_name,"class":std,"schoolname":schoolname,"section":sec},{"_id":0})

        
        match = []
        
        sub = []
        
        db_res=collection.find({"class":std,"schoolname":schoolname,"section":sec},{"_id":0})  
        
        dbs=myclient["Assignment"]
        answerCollection=dbs["submission"]
        
        
        count = 0
            
        for key in db_res:
            
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_res_student=collection_student.find_one({"regno":regno,"section":sec,"class":std,"schoolname":schoolname},{"_id":0})
            if db_res_student != None:
                if key != None:
                    
                    ans_response = answerCollection.find_one({"assign-no":key["assign-no"],"regno":regno,"class":std,"schoolname":schoolname,"section":sec},{"_id":0})
                                                          
                    if ans_response == None:
                    
#                         print(key)
                        match.append(key)
                    else:
                        

                        if ans_response["assign-no"] == key["assign-no"]:
                            count = count + 1

                        else:
                            match.append(key)
                                
                    
                                
            else:
                return json.dumps({"status":"failed","response":"student not found"})

        if len(match) == 0:
            if count > 0:
                return json.dumps({"status": "existed","response": "assignment already submitted"})
            else:
                return json.dumps({"status":"failed","response":"assignment not found"})
        else:
            return json.dumps({"status":"success","response":match})
          


In [188]:
@app.route('/exam-time-table/getAll',methods=["GET", "POST"])
def exam_time_table_getAll():
    db = myclient["Class"]
    collection = db["exam - time table"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "Time table not found","response": "Time table not found"})

In [189]:
@app.route('/teacher/logout',methods=["POST"])
def admin_portal_teacher_logout():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        regId = request.form['regId']
        passw = request.form['password']
        db_response = collection.find_one({"regId": regId,"password":passw})
        if db_response != None:
            if db_response['loggedIn'] != False:
               
                update_result = collection.update_one({"regId": regId,"password":passw,"loggedIn": True},{"$set":{"loggedIn":False}})
                if update_result.acknowledged == True:
                    response_string = {"status": "logout success","response":"ok"}

                else:
                    response_string = {"status":"failed","response":"database error"}
                return json.dumps(response_string, indent=4)
               
            else:
                return json.dumps({"status": "failed","response": "You are logged in on another device"})
        else:
            return json.dumps({"status": "failed","response": "User not found"})

In [190]:
@app.route('/assignment/submission',methods=["POST", "GET"])
def assignment_submission():
    answerCollection = myclient["Assignment"]["submission"]
    if request.method == "POST":
        
        user_document = request.get_json()
        
        submit_date = user_document['submitted_date']
        submit_time = user_document['submitted_time']
        sub_name = user_document['subject']
        std = user_document["class"]
        sec = user_document['section']
        schoolname = user_document['schoolname']
        regno = user_document['regno']
        user_document['tag'] = "final"
        
        collection = myclient["accounts"]["student"]
        
        db_response = collection.find_one({"regno":regno,"class": std,"section": sec,"schoolname":schoolname})
        
        if db_response != None:
        
#             answerCollection = myclient["Class"]["assignment-submission"]
            if answerCollection != None:
                answerCollection.insert_one(user_document)
                return make_response(json.dumps({"status": "success", "message":"Assignment Submission received"}),200)
        else:
            return make_response(json.dumps({"status": "failed", "message":"user not found"}),400)
       
    else:
        return make_response(json.dumps({"status": "failed", "message":"Forbidden Access"}),403)

In [191]:
@app.route('/print/pdf',methods=["POST"])
def print_pdf_comb():
    db=myclient["English-I-QuestionBank"]
    collection=db["EnglishQuestionPaper"]
    if request.method=='POST':
        
        user = request.form["user"]
        exam_date = request.form["date of exam"]
        test_type = request.form["type of test"]
        sub_name = request.form["title"]
        std = request.form["standard"]
        schoolname = request.form["school name"]
        
        db_response=collection.find_one({"user":user,"type of test":test_type,"school name":schoolname,"standard":std,"title":sub_name},{"_id":0})
        
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "failed","response": "Question not found"})




In [192]:
'''
GET:
    parameter: 
        regno->str
    return value:
        user_document->dict
        
'''
@app.route('/examination/review-page', methods=['POST'])
def review_page_1():
    if request.method == 'POST':
        data = request.get_json()
        print(data)
        
        exam_date = data['exam-date']
        sub_name = data['subject-name']
        std = data["class"]
        testtype = data['exam-name']
        schoolname = data['schoolname']
        regno = data['regno']
        
            
        collection = myclient["examination"]["formattedResponse-paperI"]

        if testtype == "Quarterly Examination" or testtype == "Half-yearly Examination" or testtype == "Annual Examination" or testtype == "HSC Examination" or testtype == "SSLC Examination":

            user_document = collection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

        else:
            user_document = collection.find_one({"regno": regno,"exam-date":exam_date,"class":std,"exam-name":testtype,"schoolname":schoolname},{"_id":0, "timestamp":0, "question paper id":0,"subject-name":0,"schoolname":0,"regno": 0,"exam-date":0,"class":0,"exam-name":0})

        if user_document != None:

            return make_response(json.dumps({'regno': regno, 'answer':user_document}), 200)

        else:

            return make_response(json.dumps({'regno': regno, 'message':'User document not found or processing in progress'}), 404)

    else:
        return make_response(json.dumps({'message':'Forbidden Access'}), 403)

# Tracking Learning Progress

In [193]:
@app.route('/tracking-learning-progress',methods=["GET", "POST"])
def tracking_learning_progress():
    db = myclient["Class"]
    collection = db["tracking"]
    name = request.get_json()
    myObject = {}
    if request.method == 'POST':
        db_response = collection.find_one({"regno": name['regno'],"class":name['class'],"section":name['section'],"schoolname":name['schoolname']})
        if db_response != None:
            myObject = db_response["learning-progress"]
            for j in name["learning-progress"]:
                myObject[j] = name["learning-progress"][j]
                
            update_result = collection.update_one({"_id":ObjectId(db_response['_id']),"regno": name['regno']},{"$set":{"learning-progress":myObject}})
            if update_result.acknowledged == True and update_result.modified_count == 1:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error"}
            return json.dumps(response_string, indent=4)
                
        else:
            insert_response = collection.insert_one(name)
            if insert_response.acknowledged == True:
                response_string = {"status": "success","response":"ok"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        return json.dumps(response_string, indent=4)
    return json.dumps({"status": "user not found","response": "User not found"})

In [194]:
@app.route('/student/getLearningProgress',methods=["GET", "POST"])
def student_get_learning_progress():
    db = myclient["Class"]
    collection = db["tracking"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
    return json.dumps({"status": "user not found","response": "User not found"})

# ASSIGNMENT - PRINT

In [195]:
@app.route('/assignment/print',methods=["POST"])
def print_assignment():
    db=myclient["Assignment"]
    collection=db["submission"]
    if request.method=='POST':
        
        regno = request.form["regno"]
        sub_name = request.form["subject"]
        std = request.form["class"]
        section = request.form["section"]
        schoolname = request.form["schoolname"]
        assign_no = request.form["assign-no"]
        
        db_response=collection.find_one({"regno":regno,"section":section,"schoolname":schoolname,"class":std,"subject":sub_name,"assign-no":assign_no},{"_id":0})
        
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "failed","response": f"Assignment submission for {regno} not found"})




In [196]:
@app.route('/assignment/submission/getAll',methods=["POST"])
def assign_sub_getAll():
    
    db=myclient["Assignment"]
    collection=db["submission"]
    
    if request.method=='POST':
#         data = request.get_json()
        
#         exam_date = data['end-date']
#         sub_name = data['subject']
        std = request.form["class"]
        
        schoolname = request.form['schoolname']
    
        
#         db_res=collection.find_one({"end-date":exam_date,"subject":sub_name,"class":std,"schoolname":schoolname,"section":sec},{"_id":0})

        
        match = []
        
        sub = []
        
        db_res=collection.find({"class":std,"schoolname":schoolname},{"_id":0})  
        
        
        
        count = 0
            
        for key in db_res:
            regno = key["regno"]
            sec = key["section"]
            
            db_student=myclient["accounts"]
            collection_student=db_student["student"]
            db_res_student=collection_student.find_one({"regno":regno,"section":sec,"class":std,"schoolname":schoolname},{"_id":0})
            if db_res_student != None:
                if key != None:

                    match.append(key) 
                                
            else:
                return json.dumps({"status":"failed","response":"student not found"})

        if len(match) == 0:
         
            return json.dumps({"status":"failed","response":"assignment not found"})
        else:
            return json.dumps(match)
          


In [197]:
@app.route('/update/assignmentDetails',methods=["POST"])
def update_assignmentDetails():
#   declaration space
    db = myclient["Assignment"]
    collection = db["details"]
    data = {
        "regId" : request.form["regId"],
        
#         "filename" : request.form["file_name"],
        "class" : request.form["class"],
        "section" : request.form["section"],
        "schoolname" : request.form["schoolname"],
        "subject" : request.form["subject"],
        "end-date":request.form["end-date"],
        "end-time": request.form["end-time"],
        "createdAt" : request.form["createdAt"],
        
    }
    
    
    response_string = {}
    if request.method == 'POST':
        
        db_response = collection.find_one(data)
        
        if db_response == None:
            db_res = collection.insert_one(data)

            if db_res.acknowledged == True:
                response_string = {"status": "success","response":"details inserted"}
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
        else:
            delete_file = collection.delete_one({"_id":ObjectId(db_response["_id"])})
            
            if delete_file.acknowledged == True:
                
                db_res = collection.insert_one(data)

                if db_res.acknowledged == True:
                    response_string = {"status": "success","response":"details updated"}
                else:
                    response_string = {"status":"failed","response":"database error. request not acknowledged"}
                
            else:
                response_string = {"status":"failed","response":"database error. request not acknowledged"}
                
    return json.dumps(response_string)

In [198]:
@app.route('/assignmentData/getAll',methods=["GET", "POST"])
def assignData_get():
    db = myclient["Assignment"]
    collection = db["details"]
    if request.method == 'POST':
        db_response = list(collection.find({},{"_id":0}))
        if db_response != None:
            return json.dumps(db_response)
        else:
            return json.dumps({"status": "failed","response": "Assignment not found"})

In [199]:
@app.route('/user/aadharcheck',methods=["POST"])
def user_aadhar_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        aadhar = request.form['aadhar']
        db_response = collection.find_one({"aadhar": aadhar})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"aadhar": aadhar})
            if db_response != None:
                return json.dumps({"status": "success","response": "ok"})
            
            else:
                db = myclient["accounts"]
                collection = db["admin"]
                db_response = collection.find_one({"aadhar": aadhar})
                if db_response != None:
                    return json.dumps({"status": "success","response": "ok"})
                else:
                    db = myclient["accounts"]
                    collection = db["examiner"]
                    db_response = collection.find_one({"aadhar": aadhar})
                    if db_response != None:
                        return json.dumps({"status": "success","response": "ok"})
                    else:
                        db = myclient["accounts-db"]
                        collection = db["teacher"]
                        db_response = collection.find_one({"aadhar": aadhar})
                        if db_response != None:
                            return json.dumps({"status": "success","response": "ok"})
                        else:
                            return json.dumps({"status": "failed","response": "User not found"})

In [200]:
@app.route('/user/phnocheck',methods=["POST"])
def user_phno_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        mobile = request.form['mobile']
        db_response = collection.find_one({"mobile": mobile})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"mobile": mobile})
            if db_response != None:
                return json.dumps({"status": "success","response": "ok"})
            else:
                db = myclient["accounts"]
                collection = db["admin"]
                db_response = collection.find_one({"mobile": mobile})
                if db_response != None:
                    return json.dumps({"status": "success","response": "ok"})
                else:
                    db = myclient["accounts"]
                    collection = db["examiner"]
                    db_response = collection.find_one({"mobile": mobile})
                    if db_response != None:
                        return json.dumps({"status": "success","response": "ok"})
                    else:
                        db = myclient["accounts-db"]
                        collection = db["teacher"]
                        db_response = collection.find_one({"mobile": mobile})
                        if db_response != None:
                            return json.dumps({"status": "success","response": "ok"})
                        else:
                            return json.dumps({"status": "failed","response": "User not found"})

In [201]:
@app.route('/user/mailid_check',methods=["POST"])
def user_mail_check():
    db = myclient["accounts"]
    collection = db["teacher"]
    if request.method == 'POST':
        email = request.form['email']
        db_response = collection.find_one({"email": email})
        if db_response != None:
            return json.dumps({"status": "success","response": "ok"})
        else:
            db = myclient["accounts"]
            collection = db["student"]
            db_response = collection.find_one({"email": email})
            if db_response != None:
                return json.dumps({"status": "success","response": "ok"})
            else:
                db = myclient["accounts"]
                collection = db["admin"]
                db_response = collection.find_one({"email": email})
                if db_response != None:
                    return json.dumps({"status": "success","response": "ok"})
                else:
                    db = myclient["accounts"]
                    collection = db["examiner"]
                    db_response = collection.find_one({"email": email})
                    if db_response != None:
                        return json.dumps({"status": "success","response": "ok"})
                    else:
                        db = myclient["accounts-db"]
                        collection = db["teacher"]
                        db_response = collection.find_one({"email": email})
                        if db_response != None:
                            return json.dumps({"status": "success","response": "ok"})
                        else:
                            return json.dumps({"status": "failed","response": "User not found"})

# Starting the server

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0',port=80,threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:45] "GET /teacher/sslc-enrollment HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:46] "GET /static/login-page.css HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:46] "GET /static/teacher.js HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:48] "GET /static/index.js HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:48] "POST /admin-portal/teacher-profile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:48] "GET /teacher/htmlfile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:49] "GET /static/filename.jpg HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:57] "POST /teacher/sslc-course-enrollment HTTP/1.1" 200 -


{'9': {'B': {'1': 'Tamil', '2': 'English'}}, '10': {'B': {'1': 'Tamil', '2': 'English'}}, '11': {'A': {'1': 'english', '2': 'computer-science'}}, '12': {}}


INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:59] "GET /teacher/home-page HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:59] "POST /admin-portal/teacher-profile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:21:59] "GET /teacher/htmlfile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:22:04] "GET /teacher/hsc-enrollment HTTP/1.1" 200 -
INFO:werkzeug:49.205.82.177 - - [23/Oct/2021 20:22:05] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:49.205.82.177 - - [23/Oct/2021 20:22:05] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:22:05] "POST /admin-portal/teacher-profile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:22:07] "GET /teacher/htmlfile HTTP/1.1" 200 -
INFO:werkzeug:117.202.72.9 - - [23/Oct/2021 20:22:14] "POST /teacher/sslc-course-enrollment HTTP/1.1" 200 -


{'9': {'B': {'1': 'Tamil', '2': 'English'}}, '10': {'B': {'1': 'Tamil', '2': 'English'}, 'A': {'1': 'Tamil', '2': 'English'}}, '11': {'A': {'1': 'english', '2': 'computer-science'}}, '12': {}}


...

INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:06:03] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:06:03] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:07:05] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:07:05] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:08:01] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:08:01] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:08:29] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:08:29] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:10:02] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:10:02] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 -

INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:35:12] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:35:28] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:35:29] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:36:27] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:36:27] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:38:02] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:38:02] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:38:27] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:38:27] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 06:40:12] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 -

INFO:werkzeug:212.193.30.206 - - [25/Oct/2021 07:13:17] "POST /boaform/admin/formLogin HTTP/1.1" 404 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:13:27] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:13:27] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:15:10] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:15:10] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:15:54] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:15:54] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:17:04] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:17:04] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:17:35] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.12

INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:49:05] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:49:05] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:50:08] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:50:08] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:50:59] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:50:59] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:52:02] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:52:02] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:53:01] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 07:53:01] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 -

INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:23:55] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:24:46] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:24:46] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:25:30] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:25:30] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:26:29] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:26:29] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:27:52] "POST /assignment/get HTTP/1.1" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021 08:27:52] "POST /content/get/response HTTP/1.1" 200 -
INFO:werkzeug:31.208.186.136 - - [25/Oct/2021 08:28:26] "GET / HTTP/1.0" 200 -
INFO:werkzeug:157.49.195.128 - - [25/Oct/2021